This notebook is designed for exploring and interacting with an XML file.

Contributors: Liran,

To run the notebook, please access the shared group folder at the following link:
[shared folder](https://drive.google.com/drive/folders/171k0ljqmYcgJegVlb8Y-vvAs9qukJw06?usp=sharing)
and create a shortcut to it in your Google Drive.

In [1]:
import xml.etree.ElementTree as ET
import re
import sys
import os
import json

In [2]:
project_path = os.path.dirname(os.path.realpath(os.getcwd()))
project_path

'/home/liranc6/hackAI/YedidAI'

In [3]:
sys.path.append(project_path)

from src import create_data_subset

In [4]:
xml_file_path = os.path.join(project_path, "data", "dump.he.2024-03-12.xml")

In [5]:
# Parsing the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

When scrapping papers from the category "חרבות ברזל", we might not get what we wanted.

One funny example is the page [מחזור בקבוקים](https://www.kolzchut.org.il/he/%D7%9E%D7%99%D7%97%D7%96%D7%95%D7%A8_%D7%91%D7%A7%D7%91%D7%95%D7%A7%D7%99%D7%9D),
which taged with this category but is not relevant to us.

A different problem I faced was that not every right is strongly related to this specific war. For example I can't search "חרבות ברזל" because some of the rights are general to anyone who has been harmed in any war, for example disabled soldiers.

To answer this problem I used headlines extracted from: [חרבות ברזל - זכויות במצב חירום](https://www.kolzchut.org.il/he/%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_-_%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%91%D7%9E%D7%A6%D7%91_%D7%97%D7%99%D7%A8%D7%95%D7%9D#:~:text=%D7%AA%D7%95%D7%A9%D7%91%D7%99%D7%9D%20%D7%A9%D7%A4%D7%95%D7%A0%D7%95%20%D7%90%D7%95%20%D7%94%D7%AA%D7%A4%D7%A0%D7%95%20%D7%9E%D7%91%D7%99%D7%AA%D7%9D)


In [6]:
# HTML content
html_content = """
</p>
</div></div></div>
</div><div class="toc-box noprint rs_skip"><div class="share-links"><a href="https://api.whatsapp.com/send?text=%D7%97%D7%A8%D7%91%D7%95%D7%AA%20%D7%91%D7%A8%D7%96%D7%9C%20-%20%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA%20%D7%91%D7%9E%D7%A6%D7%91%20%D7%97%D7%99%D7%A8%D7%95%D7%9D%20%D7%9E%D7%90%D7%AA%D7%A8%20%22%D7%9B%D7%9C%20%D7%96%D7%9B%D7%95%D7%AA%22%0Ahttps%3A%2F%2Fwww.kolzchut.org.il%2Fhe%2F%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_-_%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%91%D7%9E%D7%A6%D7%91_%D7%97%D7%99%D7%A8%D7%95%D7%9D%3Futm_source%3Dsharebar%26utm_medium%3Dwhatsapp" class="whatsapp" data-service="whatsapp" role="button" aria-label="שתפו בווטסאפ" title="שתפו בווטסאפ"><i aria-hidden="true" class="kzi kzi-whatsapp"></i></a><a href="https://www.facebook.com/sharer/sharer.php?u=https%3A%2F%2Fwww.kolzchut.org.il%2Fhe%2F%25D7%2597%25D7%25A8%25D7%2591%25D7%2595%25D7%25AA_%25D7%2591%25D7%25A8%25D7%2596%25D7%259C_-_%25D7%2596%25D7%259B%25D7%2595%25D7%2599%25D7%2595%25D7%25AA_%25D7%2591%25D7%259E%25D7%25A6%25D7%2591_%25D7%2597%25D7%2599%25D7%25A8%25D7%2595%25D7%259D%3Futm_source%3Dsharebar%26utm_medium%3Dfacebook" class="facebook" data-service="facebook" role="button" aria-label="שתפו בפייסבוק" title="שתפו בפייסבוק"><i aria-hidden="true" class="kzi kzi-facebook"></i></a><a href="mailto:?subject=%D7%97%D7%A9%D7%91%D7%AA%D7%99%20%D7%A9%D7%99%D7%A2%D7%A0%D7%99%D7%99%D7%9F%20%D7%90%D7%95%D7%AA%D7%9A%20-%20%D7%97%D7%A8%D7%91%D7%95%D7%AA%20%D7%91%D7%A8%D7%96%D7%9C%20-%20%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA%20%D7%91%D7%9E%D7%A6%D7%91%20%D7%97%D7%99%D7%A8%D7%95%D7%9D%20%D7%9E%D7%90%D7%AA%D7%A8%20%22%D7%9B%D7%9C%20%D7%96%D7%9B%D7%95%D7%AA%22&amp;body=https%3A%2F%2Fwww.kolzchut.org.il%2Fhe%2F%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_-_%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%91%D7%9E%D7%A6%D7%91_%D7%97%D7%99%D7%A8%D7%95%D7%9D%3Futm_source%3Dsharebar%26utm_medium%3Demail" class="email" data-service="email" role="button" aria-label="שתפו בדוא&quot;ל" title="שתפו בדוא&quot;ל"><i aria-hidden="true" class="kzi kzi-email"></i></a><a class="link" data-service="link" role="button" aria-label="שתפו קישור" title="שתפו קישור" data-clipboard-text="https://www.kolzchut.org.il/he/חרבות_ברזל_-_זכויות_במצב_חירום"><i aria-hidden="true" class="kzi kzi-link"></i></a></div>
<div class="toc-title"><a class="toggle btn btn-blue" aria-expanded="false" aria-controls=""><span class="btn-text">תוכן עניינים</span><i class="kzi kzi-chevron-down" aria-hidden="true"></i></a></div></div>
<div class="wr-note wr-tip green-border-box-with-icon clearfix"><div class="wr-note-icon icon-wrapper"><i aria-hidden="true" class="kzi kzi-article-tip"></i></div><div class="wr-note-body"><div class="header wr-note-type-text pagequality-ignore">טיפ</div><div class="text-body">מומלץ לעיין גם ב<a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%A2%D7%9C_%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות על זכויות במהלך מלחמת חרבות ברזל">רשימת שאלות ותשובות</a> בנושא הזכויות בתקופת המלחמה</div></div></div>
<div class="portal-boxes-table">
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="זכותונים_ומדריכים"></span><span class="mw-headline" id=".D7.96.D7.9B.D7.95.D7.AA.D7.95.D7.A0.D7.99.D7.9D_.D7.95.D7.9E.D7.93.D7.A8.D7.99.D7.9B.D7.99.D7.9D">זכותונים ומדריכים</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%90%D7%95_%D7%94%D7%AA%D7%A4%D7%A0%D7%95_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך למי שפונו או התפנו מבתיהם עקב מלחמת חרבות ברזל">מדריך למי שפונו או התפנו מבתיהם עקב המלחמה</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%A2%D7%A6%D7%9E%D7%90%D7%99%D7%9D_%D7%95%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך לעצמאים ובעלי עסקים בתקופת מלחמת חרבות ברזל">מדריך לעצמאים ובעלי עסקים בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך לעובדים בתקופת מלחמת חרבות ברזל">מדריך לעובדים בתקופת המלחמה</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" class="mw-redirect" title="מדריך למשרתי מילואים במלחמת חרבות ברזל">מדריך למשרתי מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%9E%D7%A9%D7%AA%D7%97%D7%A8%D7%A8%D7%99%D7%9D_%D7%9E%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך למשתחררים משירות מילואים במלחמת חרבות ברזל">מדריך למשתחררים משירות מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%96%D7%9B%D7%95%D7%AA%D7%95%D7%9F_%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%A9%D7%9C_%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="זכותון למשפחות של חטופים ונעדרים במלחמת חרבות ברזל">זכותון למשפחות החטופים והנעדרים</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99%D7%9D_%D7%91%D7%9E%D7%A1%D7%99%D7%91%D7%AA_%D7%A0%D7%95%D7%91%D7%94_%D7%95%D7%A4%D7%A1%D7%99%D7%99%D7%93%D7%90%D7%A7_%D7%95%D7%91%D7%A0%D7%99_%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D" title="מדריך לנפגעים במסיבת נובה ופסיידאק ובני משפחותיהם">מדריך לנפגעים במסיבת נובה ופסיידאק ובני משפחותיהם</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%94%D7%95%D7%A8%D7%99%D7%9D_%D7%99%D7%97%D7%99%D7%93%D7%A0%D7%99%D7%99%D7%9D_(%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%A9%D7%91%D7%A8%D7%90%D7%A9%D7%9F_%D7%94%D7%95%D7%A8%D7%94_%D7%A2%D7%A6%D7%9E%D7%90%D7%99)_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך להורים יחידניים (משפחות שבראשן הורה עצמאי) במלחמת חרבות ברזל">מדריך להורים יחידניים (משפחות שבראשן הורה עצמאי) במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%99%D7%93%D7%A2_%D7%9C%D7%A2%D7%9E%D7%95%D7%AA%D7%95%D7%AA_%D7%95%D7%9C%D7%97%D7%91%D7%A8%D7%95%D7%AA_%D7%9C%D7%AA%D7%95%D7%A2%D7%9C%D7%AA_%D7%94%D7%A6%D7%99%D7%91%D7%95%D7%A8_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מידע לעמותות ולחברות לתועלת הציבור בתקופת מלחמת חרבות ברזל">מדריך לעמותות ולחברות לתועלת הציבור</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="שאלות_ותשובות_לפי_נושאים"></span><span class="mw-headline" id=".D7.A9.D7.90.D7.9C.D7.95.D7.AA_.D7.95.D7.AA.D7.A9.D7.95.D7.91.D7.95.D7.AA_.D7.9C.D7.A4.D7.99_.D7.A0.D7.95.D7.A9.D7.90.D7.99.D7.9D">שאלות ותשובות לפי נושאים</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A0%D7%A4%D7%92%D7%A2%D7%95_%D7%A4%D7%99%D7%96%D7%99%D7%AA_%D7%90%D7%95_%D7%A0%D7%A4%D7%A9%D7%99%D7%AA_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%95%D7%9C%D7%91%D7%A0%D7%99_%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D" title="שאלות ותשובות למי שנפגעו פיזית או נפשית במהלך מלחמת חרבות ברזל ולבני משפחותיהם">שאלות ותשובות למי שנפגעו פיזית או נפשית ולבני משפחותיהם</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A7%D7%A8%D7%95%D7%91%D7%99%D7%94%D7%9D_%D7%A0%D7%94%D7%A8%D7%92%D7%95_%D7%91%D7%90%D7%99%D7%A8%D7%95%D7%A2%D7%99_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות למי שקרוביהם נהרגו באירועי מלחמת חרבות ברזל">שאלות ותשובות למשפחת שכולות עקב המלחמה</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A8%D7%9B%D7%95%D7%A9%D7%9D_%D7%A0%D7%A4%D7%92%D7%A2_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות למי שרכושם נפגע במהלך מלחמת חרבות ברזל">שאלות ותשובות למי שרכושם נפגע במלחמה</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%A2%D7%A6%D7%9E%D7%90%D7%99%D7%9D_%D7%95%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות לעצמאים ובעלי עסקים במהלך מלחמת חרבות ברזל">שאלות ותשובות לעצמאים ובעלי עסקים</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%95%D7%9C%D7%9E%D7%A2%D7%A1%D7%99%D7%A7%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות לעובדים ולמעסיקים במהלך מלחמת חרבות ברזל">שאלות ותשובות לעובדים ולמעסיקים</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%9E%D7%A9%D7%A8%D7%AA%D7%99%D7%9D_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות למי שמשרתים במילואים במהלך מלחמת חרבות ברזל">שאלות ותשובות למי שמשרתים במילואים</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%91%D7%A0%D7%95%D7%A9%D7%90%D7%99_%D7%A6%D7%A8%D7%9B%D7%A0%D7%95%D7%AA_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות בנושאי צרכנות במהלך מלחמת חרבות ברזל">שאלות ותשובות בנושאי צרכנות</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/About/faq/HarvotBarzelOuestions/Pages/default.aspx">שאלות ותשובות בנושאי הביטוח הלאומי</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://parents.education.gov.il/prhnet/gov-education/safety/iron-swords-qa">שאלות ותשובות בנושאי חינוך ולימודים</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="תושבים_שפונו_או_התפנו_מביתם"></span><span class="mw-headline" id=".D7.AA.D7.95.D7.A9.D7.91.D7.99.D7.9D_.D7.A9.D7.A4.D7.95.D7.A0.D7.95_.D7.90.D7.95_.D7.94.D7.AA.D7.A4.D7.A0.D7.95_.D7.9E.D7.91.D7.99.D7.AA.D7.9D">תושבים שפונו או התפנו מביתם</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%90%D7%95_%D7%94%D7%AA%D7%A4%D7%A0%D7%95_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך למי שפונו או התפנו מבתיהם עקב מלחמת חרבות ברזל">מדריך למי שפונו או התפנו מבתיהם עקב מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%90%D7%91%D7%98%D7%9C%D7%94_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%A9%D7%90%D7%99%D7%A0%D7%9D_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%95%D7%A9%D7%9E%D7%99%D7%A6%D7%95_%D7%90%D7%AA_%D7%96%D7%9B%D7%90%D7%95%D7%AA%D7%9D_%D7%9C%D7%93%D7%9E%D7%99_%D7%90%D7%91%D7%98%D7%9C%D7%94" title="מענק אבטלה למפונים שאינם עובדים ושמיצו את זכאותם לדמי אבטלה">מענק אבטלה למפונים שאינם עובדים ושמיצו את זכאותם לדמי אבטלה</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9E%D7%99%D7%95%D7%97%D7%93_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%91%D7%92%D7%99%D7%9C_67_%D7%95%D7%9E%D7%A2%D7%9C%D7%94_%D7%A9%D7%90%D7%99%D7%A0%D7%9D_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D" title="מענק מיוחד למפונים בגיל 67 ומעלה שאינם עובדים">מענק מיוחד למפונים בגיל 67 ומעלה שאינם עובדים</a></li>
<li><a href="/he/%D7%A1%D7%99%D7%95%D7%A2_%D7%91%D7%94%D7%A9%D7%90%D7%9C%D7%94_%D7%95%D7%91%D7%A8%D7%9B%D7%99%D7%A9%D7%AA_%D7%A2%D7%96%D7%A8%D7%99%D7%9D_%D7%9C%D7%93%D7%99%D7%95%D7%A8_%D7%9C%D7%90%D7%A0%D7%A9%D7%99%D7%9D_%D7%A2%D7%9D_%D7%9E%D7%95%D7%92%D7%91%D7%9C%D7%95%D7%AA_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="סיוע בהשאלה וברכישת עזרים לדיור לאנשים עם מוגבלות שפונו מבתיהם במלחמת חרבות ברזל">סיוע בהשאלה וברכישת עזרים לדיור לאנשים עם מוגבלות שפונו מבתיהם</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/evacuees-details-form">בקשה לפינוי למלון במימון המדינה לתושבי קריית שמונה</a></li>
<li><a href="/he/%D7%94%D7%92%D7%A9%D7%AA_%D7%91%D7%A7%D7%A9%D7%94_%D7%9E%D7%A7%D7%95%D7%95%D7%A0%D7%AA_%D7%9C%D7%A9%D7%99%D7%A0%D7%95%D7%99_%D7%9B%D7%AA%D7%95%D7%91%D7%AA_%D7%9E%D7%92%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%99%D7%99%D7%A9%D7%95%D7%91_%D7%9E%D7%A4%D7%95%D7%A0%D7%94_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הגשת בקשה מקוונת לשינוי כתובת מגורים ליישוב מפונה במלחמת חרבות ברזל">הגשת בקשה מקוונת לשינוי כתובת מגורים ליישוב מפונה</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%A2%D7%99%D7%93%D7%95%D7%93_%D7%97%D7%96%D7%A8%D7%94_%D7%9C%D7%A2%D7%91%D7%95%D7%93%D7%94_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9E%D7%A7%D7%95%D7%9D_%D7%9E%D7%92%D7%95%D7%A8%D7%99%D7%94%D7%9D_%D7%90%D7%95_%D7%9E%D7%A7%D7%95%D7%9D_%D7%A2%D7%91%D7%95%D7%93%D7%AA%D7%9D_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91%D7%99%D7%9D_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק עידוד חזרה לעבודה לעובדים שמקום מגוריהם או מקום עבודתם ביישובים שפונו במלחמת חרבות ברזל">מענק עידוד חזרה לעבודה לעובדים שמקום מגוריהם או מקום עבודתם ביישובים שפונו במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9E%D7%AA%D7%92%D7%95%D7%A8%D7%A8%D7%99%D7%9D_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91%D7%99%D7%9D_%D7%A9%D7%94%D7%AA%D7%A4%D7%A0%D7%95_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="איסור פיטורי עובדים שמתגוררים ביישובים שהתפנו במהלך מלחמת חרבות ברזל">איסור פיטורי עובדים שמתגוררים ביישובים שהתפנו</a></li>
<li><a href="/he/%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%A9%D7%9B%D7%A8_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A0%D7%A2%D7%93%D7%A8%D7%95_%D7%9E%D7%A2%D7%91%D7%95%D7%93%D7%AA%D7%9D_%D7%91%D7%A9%D7%9C_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="תשלום שכר לעובדים שנעדרו מעבודתם בשל מלחמת חרבות ברזל">תשלום שכר לעובדים שנעדרו מעבודתם בשל מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A4%D7%98%D7%95%D7%A8_%D7%9E%D7%90%D7%A8%D7%A0%D7%95%D7%A0%D7%94_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%90%D7%95_%D7%94%D7%AA%D7%A4%D7%A0%D7%95_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פטור מארנונה למי שפונו או התפנו מבתיהם עקב מלחמת חרבות ברזל">פטור מארנונה למי שפונו או התפנו מבתיהם עקב מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A0%D7%97%D7%94_%D7%91%D7%90%D7%A8%D7%A0%D7%95%D7%A0%D7%94_%D7%91%D7%99%D7%A9%D7%95%D7%91%D7%99_%D7%A2%D7%95%D7%98%D7%A3_%D7%A2%D7%96%D7%94" title="הנחה בארנונה בישובי עוטף עזה">הנחה בארנונה בישובי עוטף עזה</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%90%D7%9B%D7%9C%D7%95%D7%A1_%D7%9C%D7%AA%D7%95%D7%A9%D7%91%D7%99%D7%9D_%D7%A9%D7%94%D7%AA%D7%A4%D7%A0%D7%95_%D7%9E%D7%91%D7%99%D7%AA%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק אכלוס לתושבים שהתפנו מביתם בתקופת מלחמת חרבות ברזל">מענק אכלוס לתושבים שהתפנו מביתם</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%94%D7%AA%D7%90%D7%A8%D7%92%D7%A0%D7%95%D7%AA_%D7%9C%D7%AA%D7%95%D7%A9%D7%91%D7%99%D7%9D_%D7%A9%D7%94%D7%AA%D7%A4%D7%A0%D7%95_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק התארגנות לתושבים שהתפנו מבתיהם בתקופת מלחמת חרבות ברזל">מענק התארגנות לתושבים שהתפנו מבתיהם בדרום ובצפון הארץ</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%91%D7%A0%D7%A7%D7%99%D7%9D_%D7%95%D7%91%D7%97%D7%91%D7%A8%D7%95%D7%AA_%D7%94%D7%90%D7%A9%D7%A8%D7%90%D7%99_%D7%9C%D7%AA%D7%95%D7%A9%D7%91%D7%99_%D7%94%D7%93%D7%A8%D7%95%D7%9D,_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D,_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%94%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%95%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%94%D7%94%D7%A8%D7%95%D7%92%D7%99%D7%9D,_%D7%94%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%94%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" class="mw-redirect" title="הקלות בבנקים ובחברות האשראי לתושבי הדרום, למפונים מבתיהם, למשרתי המילואים ולמשפחות ההרוגים, החטופים והנעדרים במלחמת חרבות ברזל">הקלות בבנקים ובחברות האשראי</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלומים לרשויות שונות בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלומים לרשויות שונות בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית פעולה שיש לבצע לפי פסק דין בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית פעולה שיש לבצע לפי פסק דין בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלום או פעולה שיש לבצע לפי חוזה החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלומים לרשויות שונות בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלומים לרשויות שונות החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית פעולה שיש לבצע לפי פסק דין בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית פעולה שיש לבצע לפי פסק דין החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%9E%D7%A7%D7%93%D7%9E%D7%95%D7%AA_%D7%9C%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91%D7%99_%D7%A1%D7%A4%D7%A8_%D7%91%D7%93%D7%A8%D7%95%D7%9D_%D7%95%D7%91%D7%A6%D7%A4%D7%95%D7%9F_%D7%94%D7%90%D7%A8%D7%A5_%D7%91%D7%A9%D7%9C_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="תשלום מקדמות לבעלי עסקים ביישובי ספר בדרום ובצפון הארץ בשל מלחמת חרבות ברזל">תשלום מקדמות לבעלי עסקים ביישובי ספר בדרום ובצפון הארץ</a></li>
<li><a href="/he/%D7%A7%D7%91%D7%95%D7%A8%D7%94_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="קבורה במהלך מלחמת חרבות ברזל">קבורה זמנית או קבועה ביישובים אחרים</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%9E%D7%A2%D7%A1%D7%99%D7%A7%D7%99%D7%9D_%D7%A9%D7%A9%D7%99%D7%9C%D7%9E%D7%95_%D7%A9%D7%9B%D7%A8_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D_%D7%95%D7%A0%D7%A2%D7%93%D7%A8%D7%95_%D7%9E%D7%94%D7%A2%D7%91%D7%95%D7%93%D7%94_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי למעסיקים ששילמו שכר לעובדים שפונו מבתיהם ונעדרו מהעבודה במלחמת חרבות ברזל">פיצוי למעסיקים ששילמו שכר לעובדים שפונו ונעדרו מהעבודה</a></li>
<li><a href="/he/%D7%A6%D7%91%D7%99%D7%A8%D7%AA_%D7%99%D7%9E%D7%99_%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9C%D7%90_%D7%99%D7%9B%D7%9C%D7%95_%D7%9C%D7%A0%D7%A6%D7%9C_%D7%90%D7%AA_%D7%99%D7%9E%D7%99_%D7%94%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל">צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל</a>.</li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%94%D7%95%D7%A6%D7%90%D7%94_%D7%9C%D7%A4%D7%95%D7%A2%D7%9C_%D7%95%D7%91%D7%9E%D7%A8%D7%9B%D7%96_%D7%9C%D7%92%D7%91%D7%99%D7%99%D7%AA_%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות עקב מלחמת חרבות ברזל">הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="http://my.gov.il/lifeevent/ironswords">פורטל "יחד" לקבלת סיוע ממשרדי הממשלה למפונים</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://ironswords.health.gov.il/rights-of-evacuated-persons/">מוקד קול הבריאות</a> למידע על קבלת שירותי בריאות <span class="phonenum phonenum-unlinked">*5400</span> או <span class="phonenum phonenum-unlinked">08-6241010</span> שלוחה 4</li>
<li>מוקד רשות האוכלוסין לתושבים שפונו ומעסיקים עובד זר בסיעוד <span class="phonenum phonenum-unlinked">074-7086100</span></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/life-events/information-for-evacuation/various-services">מידע ושירות למפונים באתר השירותים והמידע הממשלתי</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/topics/pt-for-evacuation/govil-landing-page">זכויות והטבות בנושא תחבורה והתניידות</a> באתר משרד התחבורה</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://ironswords.health.gov.il/rights-of-evacuated-persons/">שירותי בריאות למפונים ועקורים באתר משרד הבריאות</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.kivunimrights.com/">סיוע במימוש זכויות של מי שנפגעו מהמלחמה או מפעולות האיבה</a> מוקד ערב"ה בשיתוף מערך הדיגיטל הלאומי</li>
<li><a href="/he/%D7%97%D7%9E%22%D7%9C_%D7%94%D7%9E%D7%A1%D7%99%D7%99%D7%A2" title="חמ&quot;ל המסייע">חמ"ל המסייע</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="פגיעות_פיסיות_ונפשיות"></span><span class="mw-headline" id=".D7.A4.D7.92.D7.99.D7.A2.D7.95.D7.AA_.D7.A4.D7.99.D7.A1.D7.99.D7.95.D7.AA_.D7.95.D7.A0.D7.A4.D7.A9.D7.99.D7.95.D7.AA">פגיעות פיסיות ונפשיות</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A0%D7%A4%D7%92%D7%A2%D7%95_%D7%A4%D7%99%D7%96%D7%99%D7%AA_%D7%90%D7%95_%D7%A0%D7%A4%D7%A9%D7%99%D7%AA_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%95%D7%9C%D7%91%D7%A0%D7%99_%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D" title="שאלות ותשובות למי שנפגעו פיזית או נפשית במהלך מלחמת חרבות ברזל ולבני משפחותיהם">שאלות ותשובות למי שנפגעו פיזית או נפשית ולבני משפחותיהם</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99%D7%9D_%D7%91%D7%9E%D7%A1%D7%99%D7%91%D7%AA_%D7%A0%D7%95%D7%91%D7%94_%D7%95%D7%A4%D7%A1%D7%99%D7%99%D7%93%D7%90%D7%A7_%D7%95%D7%91%D7%A0%D7%99_%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D" title="מדריך לנפגעים במסיבת נובה ופסיידאק ובני משפחותיהם">מדריך לנפגעים במסיבת נובה ופסיידאק ובני משפחותיהם</a></li>
<li><a href="/he/%D7%9E%D7%A7%D7%93%D7%9E%D7%95%D7%AA_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99%D7%9D_%D7%91%D7%90%D7%99%D7%A8%D7%95%D7%A2%D7%99%D7%9D_%D7%A9%D7%9C_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מקדמות לנפגעים באירועים של מלחמת חרבות ברזל">מקדמות לנפגעים באירועים של מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלומים לרשויות שונות בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלומים לרשויות שונות בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית פעולה שיש לבצע לפי פסק דין בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית פעולה שיש לבצע לפי פסק דין בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלומים לרשויות שונות בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלומים לרשויות שונות החל מ-01.01.2024</a></li>
<li><b><a href="/he/%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%A4%D7%A2%D7%95%D7%9C%D7%95%D7%AA_%D7%90%D7%99%D7%91%D7%94" title="נפגעי פעולות איבה">נפגעי פעולות איבה</a></b></li>
<li><b><a href="/he/%D7%A0%D7%9B%D7%99_%D7%A6%D7%94%22%D7%9C_%D7%95%D7%9B%D7%95%D7%97%D7%95%D7%AA_%D7%94%D7%91%D7%99%D7%98%D7%97%D7%95%D7%9F" title="נכי צה&quot;ל וכוחות הביטחון">נכי צה"ל וכוחות הביטחון</a></b></li>
<li><b><a href="/he/%D7%9E%D7%A8%D7%9B%D7%96%D7%99%D7%9D_%D7%9C%D7%AA%D7%9E%D7%99%D7%9B%D7%94_%D7%A0%D7%A4%D7%A9%D7%99%D7%AA_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%98%D7%A8%D7%90%D7%95%D7%9E%D7%94_%D7%95%D7%97%D7%A8%D7%93%D7%94" title="מרכזים לתמיכה נפשית לנפגעי טראומה וחרדה">מרכזים לתמיכה נפשית לנפגעי טראומה וחרדה</a></b></li>
<li><a href="/he/%D7%9E%D7%95%D7%A7%D7%93%D7%99%D7%9D_%D7%9C%D7%A1%D7%99%D7%95%D7%A2_%D7%A0%D7%A4%D7%A9%D7%99_%D7%9E%D7%98%D7%A2%D7%9D_%D7%A7%D7%95%D7%A4%D7%95%D7%AA_%D7%94%D7%97%D7%95%D7%9C%D7%99%D7%9D_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%98%D7%A8%D7%90%D7%95%D7%9E%D7%94_%D7%95%D7%97%D7%A8%D7%93%D7%94_%D7%91%D7%A2%D7%A7%D7%91%D7%95%D7%AA_%D7%94%D7%9E%D7%A6%D7%91_%D7%94%D7%91%D7%98%D7%97%D7%95%D7%A0%D7%99" title="מוקדים לסיוע נפשי מטעם קופות החולים לנפגעי טראומה וחרדה בעקבות המצב הבטחוני">מוקדים לסיוע נפשי מטעם קופות החולים לנפגעי טראומה וחרדה בעקבות המצב הבטחוני</a></li>
<li><a href="/he/%D7%9E%D7%A8%D7%9B%D7%96%D7%99_%D7%9E%D7%99%D7%A6%D7%95%D7%99_%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%91%D7%91%D7%AA%D7%99_%D7%97%D7%95%D7%9C%D7%99%D7%9D_%D7%A2%D7%91%D7%95%D7%A8_%D7%9E%D7%98%D7%95%D7%A4%D7%9C%D7%99%D7%9D_%D7%95%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D" title="מרכזי מיצוי זכויות בבתי חולים עבור מטופלים ומשפחותיהם">מרכזי מיצוי זכויות בבתי חולים עבור מטופלים ומשפחותיהם</a></li>
<li><a href="/he/%D7%98%D7%99%D7%A4%D7%95%D7%9C_%D7%A0%D7%A4%D7%A9%D7%99_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%97%D7%A8%D7%93%D7%94_%D7%A2%D7%A7%D7%91_%D7%A4%D7%A2%D7%95%D7%9C%D7%AA_%D7%90%D7%99%D7%91%D7%94" title="טיפול נפשי לנפגעי חרדה עקב פעולת איבה">טיפול נפשי לנפגעי חרדה עקב פעולת איבה</a></li>
<li><a href="/he/%D7%96%D7%9B%D7%95%D7%AA%D7%95%D7%9F_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%AA%D7%A7%D7%99%D7%A4%D7%95%D7%AA_%D7%92%D7%96%D7%A2%D7%A0%D7%99%D7%95%D7%AA_%D7%95%D7%9C%D7%90%D7%95%D7%9E%D7%A0%D7%99%D7%95%D7%AA" title="זכותון לנפגעי תקיפות גזעניות ולאומניות">זכותון לנפגעי תקיפות גזעניות ולאומניות</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://ironswords.health.gov.il/rights-of-victims/">מידע, מוקדי חירום ואיתור מאושפזים</a> באתר משרד הבריאות</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/benefits/Victims_of_Hostilities/Pages/MerkazeiSiua.aspx">מרכזי סיוע לנפגעי חרדה</a> באתר המוסד לביטוח לאומי</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://ironswords.health.gov.il/coping-with-emergency/">מידע וכלים להתמודדות נפשית במצבי חירום</a> באתר משרד הבריאות</li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.kivunimrights.com/">סיוע במימוש זכויות של מי שנפגעו מהמלחמה או מפעולות האיבה</a> מוקד ערב"ה בשיתוף מערך הדיגיטל הלאומי</li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.sheba.co.il/%D7%98%D7%95%D7%A4%D7%A1/%D7%A9%D7%99%D7%91%D7%90/198899">טיפול נפשי מקוון למבוגרים וילדים</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="עובדים"></span><span class="mw-headline" id=".D7.A2.D7.95.D7.91.D7.93.D7.99.D7.9D">עובדים</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך לעובדים בתקופת מלחמת חרבות ברזל">מדריך לעובדים בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%95%D7%9C%D7%9E%D7%A2%D7%A1%D7%99%D7%A7%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות לעובדים ולמעסיקים במהלך מלחמת חרבות ברזל">שאלות ותשובות לעובדים ולמעסיקים</a></li>
<li><a href="/he/%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%A9%D7%9B%D7%A8_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A0%D7%A2%D7%93%D7%A8%D7%95_%D7%9E%D7%A2%D7%91%D7%95%D7%93%D7%AA%D7%9D_%D7%91%D7%A9%D7%9C_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="תשלום שכר לעובדים שנעדרו מעבודתם בשל מלחמת חרבות ברזל">תשלום שכר לעובדים שנעדרו מעבודתם בשל מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%90%D7%91%D7%98%D7%9C%D7%94_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%A9%D7%90%D7%99%D7%A0%D7%9D_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%95%D7%A9%D7%9E%D7%99%D7%A6%D7%95_%D7%90%D7%AA_%D7%96%D7%9B%D7%90%D7%95%D7%AA%D7%9D_%D7%9C%D7%93%D7%9E%D7%99_%D7%90%D7%91%D7%98%D7%9C%D7%94" title="מענק אבטלה למפונים שאינם עובדים ושמיצו את זכאותם לדמי אבטלה">מענק אבטלה למפונים שאינם עובדים ושמיצו את זכאותם לדמי אבטלה</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9E%D7%99%D7%95%D7%97%D7%93_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%91%D7%92%D7%99%D7%9C_67_%D7%95%D7%9E%D7%A2%D7%9C%D7%94_%D7%A9%D7%90%D7%99%D7%A0%D7%9D_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D" title="מענק מיוחד למפונים בגיל 67 ומעלה שאינם עובדים">מענק מיוחד למפונים בגיל 67 ומעלה שאינם עובדים</a></li>
<li><a href="/he/%D7%93%D7%9E%D7%99_%D7%90%D7%91%D7%98%D7%9C%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="דמי אבטלה בתקופת מלחמת חרבות ברזל">דמי אבטלה בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%AA%D7%9E%D7%A8%D7%95%D7%A5_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%94%D7%97%D7%9C%D7%95_%D7%9C%D7%A2%D7%91%D7%95%D7%93_%D7%91%D7%A2%D7%A0%D7%A3_%D7%94%D7%97%D7%A7%D7%9C%D7%90%D7%95%D7%AA_%D7%95%D7%94%D7%91%D7%A0%D7%99%D7%99%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק תמרוץ לעובדים שהחלו לעבוד בענף החקלאות והבנייה בתקופת מלחמת חרבות ברזל">מענק תמרוץ לעובדים שהחלו לעבוד בענף החקלאות והבנייה בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%A2%D7%99%D7%93%D7%95%D7%93_%D7%97%D7%96%D7%A8%D7%94_%D7%9C%D7%A2%D7%91%D7%95%D7%93%D7%94_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9E%D7%A7%D7%95%D7%9D_%D7%9E%D7%92%D7%95%D7%A8%D7%99%D7%94%D7%9D_%D7%90%D7%95_%D7%9E%D7%A7%D7%95%D7%9D_%D7%A2%D7%91%D7%95%D7%93%D7%AA%D7%9D_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91%D7%99%D7%9D_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק עידוד חזרה לעבודה לעובדים שמקום מגוריהם או מקום עבודתם ביישובים שפונו במלחמת חרבות ברזל">מענק עידוד חזרה לעבודה לעובדים שמקום מגוריהם או מקום עבודתם ביישובים שפונו במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A2%D7%A1%D7%A7%D7%AA_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%91%D7%A9%D7%A2%D7%95%D7%AA_%D7%A0%D7%95%D7%A1%D7%A4%D7%95%D7%AA_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="העסקת עובדים בשעות נוספות במהלך מלחמת חרבות ברזל">העסקת עובדים בשעות נוספות במהלך מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9E%D7%99%D7%95%D7%97%D7%93_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%9E%D7%A2%D7%9C_%D7%92%D7%99%D7%9C_67_%D7%A9%D7%94%D7%95%D7%A6%D7%90%D7%95_%D7%9C%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%9C%D7%90_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%95%D7%98%D7%A8%D7%95_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק מיוחד לעובדים מעל גיל 67 שהוצאו לחופשה ללא תשלום או פוטרו בתקופת מלחמת חרבות ברזל">מענק מיוחד לעובדים מעל גיל 67 שהוצאו לחופשה ללא תשלום או פוטרו</a></li>
<li><a href="/he/%D7%94%D7%A4%D7%A8%D7%A9%D7%95%D7%AA_%D7%9C%D7%91%D7%99%D7%98%D7%95%D7%97_%D7%A4%D7%A0%D7%A1%D7%99%D7%95%D7%A0%D7%99_%D7%95%D7%9C%D7%A7%D7%A8%D7%9F_%D7%94%D7%A9%D7%AA%D7%9C%D7%9E%D7%95%D7%AA_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="הפרשות לביטוח פנסיוני ולקרן השתלמות לעובדים במילואים">הפרשות לביטוח פנסיוני ולקרן השתלמות לעובדים במילואים</a></li>
<li><a href="/he/%D7%A6%D7%95_%D7%92%D7%99%D7%95%D7%A1_%D7%9C%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%A2%D7%91%D7%95%D7%93%D7%94_%D7%91%D7%9E%D7%A4%D7%A2%D7%9C%D7%99%D7%9D_%D7%97%D7%99%D7%95%D7%A0%D7%99%D7%99%D7%9D_%D7%90%D7%95_%D7%91%D7%9E%D7%A4%D7%A2%D7%9C%D7%99%D7%9D_%D7%9C%D7%9E%D7%AA%D7%9F_%D7%A9%D7%99%D7%A8%D7%95%D7%AA%D7%99%D7%9D_%D7%A7%D7%99%D7%95%D7%9E%D7%99%D7%99%D7%9D" title="צו גיוס לשירות עבודה במפעלים חיוניים או במפעלים למתן שירותים קיומיים">צו גיוס לשירות עבודה במפעלים חיוניים או במפעלים למתן שירותים קיומיים</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/Departments/General/war-faq">חרבות ברזל: שאלות ותשובות בנושא זכויות עובדים</a> באתר משרד העבודה</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/faq/faq-emergency">שאלות ותשובות בנושא יחסי עבודה בשעת חירום</a> באתר משרד העבודה</li>
<li><a href="/he/%D7%94%D7%A4%D7%A7%D7%93%D7%95%D7%AA_%D7%9C%D7%91%D7%99%D7%98%D7%95%D7%97_%D7%A4%D7%A0%D7%A1%D7%99%D7%95%D7%A0%D7%99_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A0%D7%97%D7%98%D7%A4%D7%95_%D7%90%D7%95_%D7%94%D7%95%D7%9B%D7%A8%D7%96%D7%95_%D7%9B%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הפקדות לביטוח פנסיוני לעובדים שנחטפו או הוכרזו כנעדרים במלחמת חרבות ברזל">הפקדות לביטוח פנסיוני לעובדים שנחטפו או הוכרזו כנעדרים</a></li>
<li>איסור פיטורי עובדים:
<ul><li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A0%D7%A2%D7%93%D7%A8%D7%95_%D7%9E%D7%A2%D7%91%D7%95%D7%93%D7%AA%D7%9D_%D7%9C%D7%A6%D7%95%D7%A8%D7%9A_%D7%94%D7%A9%D7%92%D7%97%D7%94_%D7%A2%D7%9C_%D7%99%D7%9C%D7%93%D7%99%D7%94%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="איסור פיטורי עובדים שנעדרו מעבודתם לצורך השגחה על ילדיהם במהלך מלחמת חרבות ברזל">עובדים שנעדרו מעבודתם לצורך השגחה על ילדיהם</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%94%D7%9D_%D7%90%D7%95_%D7%91%D7%A0%D7%99_%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D_%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%90%D7%95_%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="איסור פיטורי עובדים שהם או בני משפחותיהם חטופים או נעדרים במהלך מלחמת חרבות ברזל">עובדים שהם או בני משפחותיהם חטופים או נעדרים</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9E%D7%AA%D7%92%D7%95%D7%A8%D7%A8%D7%99%D7%9D_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91%D7%99%D7%9D_%D7%A9%D7%94%D7%AA%D7%A4%D7%A0%D7%95_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="איסור פיטורי עובדים שמתגוררים ביישובים שהתפנו במהלך מלחמת חרבות ברזל">עובדים שמתגוררים ביישובים שהתפנו</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A0%D7%A2%D7%93%D7%A8%D7%95_%D7%9E%D7%A2%D7%91%D7%95%D7%93%D7%AA%D7%9D_%D7%A2%D7%A7%D7%91_%D7%94%D7%A0%D7%97%D7%99%D7%95%D7%AA_%D7%A4%D7%99%D7%A7%D7%95%D7%93_%D7%94%D7%A2%D7%95%D7%A8%D7%A3_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="איסור פיטורי עובדים שנעדרו מעבודתם עקב הנחיות פיקוד העורף במהלך מלחמת חרבות ברזל">עובדים שנעדרו מעבודתם עקב הנחיות פיקוד העורף</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%90%D7%95_%D7%91%D7%AA%D7%95%D7%9D_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="איסור פיטורי עובד במהלך או בתום שירות מילואים">עובדים במהלך שירות מילואים ולאחריו</a></li></ul></li>
<li><a href="/he/%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%91%D7%99%D7%95%D7%96%D7%9E%D7%AA_%D7%94%D7%9E%D7%A2%D7%A1%D7%99%D7%A7" title="חופשה ביוזמת המעסיק">מגבלות להוצאה לחופשה של עובד במהלך מצב חירום או במצב מיוחד בעורף</a></li>
<li><a href="/he/%D7%A6%D7%91%D7%99%D7%A8%D7%AA_%D7%99%D7%9E%D7%99_%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9C%D7%90_%D7%99%D7%9B%D7%9C%D7%95_%D7%9C%D7%A0%D7%A6%D7%9C_%D7%90%D7%AA_%D7%99%D7%9E%D7%99_%D7%94%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל">צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9C%D7%A2%D7%95%D7%91%D7%93_%D7%A9%D7%94%D7%95%D7%A6%D7%90_%D7%9C%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%9C%D7%90_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%A9%D7%9C_%D7%91%D7%9F_%D7%96%D7%95%D7%92%D7%95_%D7%90%D7%95_%D7%94%D7%95%D7%A8%D7%94_%D7%9E%D7%A9%D7%95%D7%AA%D7%A3_%D7%9C%D7%99%D7%9C%D7%93%D7%95_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק לעובד שהוצא לחופשה ללא תשלום בתקופת מילואים של בן זוגו או הורה משותף לילדו במלחמת חרבות ברזל">מענק לעובד שהוצא לחופשה ללא תשלום בתקופת מילואים של בן זוגו או הורה משותף לילדו במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93_%D7%91%D7%A9%D7%9C_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%A9%D7%9C_%D7%91%D7%9F_%D7%96%D7%95%D7%92%D7%95_%D7%90%D7%95_%D7%A9%D7%9C_%D7%94%D7%94%D7%95%D7%A8%D7%94_%D7%94%D7%A0%D7%95%D7%A1%D7%A3_%D7%A9%D7%9C_%D7%99%D7%9C%D7%93%D7%95" title="איסור פיטורי עובד בשל שירות מילואים של בן זוגו או של ההורה הנוסף של ילדו">איסור פיטורי עובד בשל שירות מילואים של בן זוגו או של ההורה הנוסף של ילדו</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%94%D7%95%D7%A6%D7%90%D7%94_%D7%9C%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%9C%D7%90_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%A9%D7%9C_%D7%A2%D7%95%D7%91%D7%93_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%A9%D7%9C_%D7%91%D7%9F_%D7%94%D7%96%D7%95%D7%92_%D7%90%D7%95_%D7%94%D7%95%D7%A8%D7%94_%D7%9E%D7%A9%D7%95%D7%AA%D7%A3" title="איסור הוצאה לחופשה ללא תשלום של עובד במהלך שירות מילואים של בן הזוג או הורה משותף">איסור הוצאה לחופשה ללא תשלום של עובד במהלך שירות מילואים של בן הזוג או הורה משותף</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.histadrut.org.il/מוקד_המידע_והשירות">מוקד מידע לעובדים מטעם ההסתדרות החדשה</a> <span class="phonenum phonenum-unlinked">*2383</span></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="עצמאים,_חברות_ועמותות"></span><span class="mw-headline" id=".D7.A2.D7.A6.D7.9E.D7.90.D7.99.D7.9D.2C_.D7.97.D7.91.D7.A8.D7.95.D7.AA_.D7.95.D7.A2.D7.9E.D7.95.D7.AA.D7.95.D7.AA">עצמאים, חברות ועמותות</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%A2%D7%A6%D7%9E%D7%90%D7%99%D7%9D_%D7%95%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך לעצמאים ובעלי עסקים בתקופת מלחמת חרבות ברזל">מדריך לעצמאים ובעלי עסקים בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%A2%D7%A6%D7%9E%D7%90%D7%99%D7%9D_%D7%95%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות לעצמאים ובעלי עסקים במהלך מלחמת חרבות ברזל">שאלות ותשובות לעצמאים ובעלי עסקים</a></li>
<li><b><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%A2%D7%9C_%D7%94%D7%A4%D7%A1%D7%93%D7%99%D7%9D_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי לבעלי עסקים על הפסדים עקב מלחמת חרבות ברזל">פיצוי לבעלי עסקים על הפסדים עקב מלחמת חרבות ברזל</a></b>
<ul><li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99%D7%99%D7%9D_%D7%9C%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%99%D7%A9%D7%95%D7%91%D7%99%D7%9D_%D7%94%D7%9E%D7%91%D7%95%D7%A1%D7%A1%D7%99%D7%9D_%D7%A2%D7%9C_%D7%AA%D7%99%D7%99%D7%A8%D7%95%D7%AA_%D7%95%D7%99%D7%A9%D7%95%D7%91%D7%99%D7%9D_%D7%A2%D7%9D_%D7%94%D7%92%D7%91%D7%9C%D7%95%D7%AA_%D7%97%D7%9E%D7%95%D7%A8%D7%95%D7%AA,_%D7%A2%D7%9C_%D7%94%D7%A4%D7%A1%D7%93%D7%99%D7%9D_%D7%91-_2024_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצויים לעסקים בישובים המבוססים על תיירות וישובים עם הגבלות חמורות, על הפסדים ב- 2024 עקב מלחמת חרבות ברזל">פיצויים לעסקים בישובים המבוססים על תיירות וישובים עם הגבלות חמורות, על הפסדים ב- 2024 עקב מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%A7%D7%98%D7%A0%D7%99%D7%9D_%D7%A2%D7%9D_%D7%9E%D7%97%D7%96%D7%95%D7%A8_%D7%A2%D7%93_300,000_%D7%A9%22%D7%97_%D7%A2%D7%9C_%D7%94%D7%A4%D7%A1%D7%93%D7%99%D7%9D_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_(%D7%9E%D7%A2%D7%A0%D7%A7_%D7%94%D7%9E%D7%A9%D7%9B%D7%99%D7%95%D7%AA_%D7%A2%D7%A1%D7%A7%D7%99%D7%AA)" title="פיצוי לעסקים קטנים עם מחזור עד 300,000 ש&quot;ח על הפסדים עקב מלחמת חרבות ברזל (מענק המשכיות עסקית)">פיצויים לעסקים בכל הארץ עם מחזור עד 300,000 ש"ח</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%A2%D7%9D_%D7%9E%D7%97%D7%96%D7%95%D7%A8_%D7%A9%D7%91%D7%99%D7%9F_300,000_-_400_%D7%9E%D7%99%D7%9C%D7%99%D7%95%D7%9F_%D7%A9%22%D7%97_%D7%A2%D7%9C_%D7%94%D7%A4%D7%A1%D7%93%D7%99%D7%9D_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_(%D7%9E%D7%A2%D7%A0%D7%A7_%D7%94%D7%9E%D7%A9%D7%9B%D7%99%D7%95%D7%AA_%D7%A2%D7%A1%D7%A7%D7%99%D7%AA)" title="פיצוי לעסקים עם מחזור שבין 300,000 - 400 מיליון ש&quot;ח על הפסדים עקב מלחמת חרבות ברזל (מענק המשכיות עסקית)">פיצויים לעסקים בכל הארץ עם מחזור מעל 300,000 ש"ח</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9E%D7%95%D7%92%D7%93%D7%9C_%D7%A2%D7%9C_%D7%94%D7%A4%D7%A1%D7%93%D7%99%D7%9D_%D7%9C%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%90%D7%96%D7%95%D7%A8_%D7%94%D7%9E%D7%99%D7%95%D7%97%D7%93_%D7%A9%D7%91%D7%95_%D7%94%D7%95%D7%98%D7%9C%D7%95_%D7%94%D7%92%D7%91%D7%9C%D7%95%D7%AA_%D7%9E%D7%97%D7%9E%D7%99%D7%A8%D7%95%D7%AA_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי מוגדל על הפסדים לעסקים באזור המיוחד שבו הוטלו הגבלות מחמירות במלחמת חרבות ברזל">פיצוי מוגדל על הפסדים לעסקים באזור המיוחד שבו הוטלו הגבלות מחמירות</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99%D7%99%D7%9D_%D7%9C%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%99%D7%A9%D7%95%D7%91%D7%99_%D7%A1%D7%A4%D7%A8_%D7%91%D7%92%D7%99%D7%9F_%D7%94%D7%A4%D7%A1%D7%93_%D7%90%D7%95_%D7%9E%D7%A0%D7%99%D7%A2%D7%AA_%D7%A8%D7%95%D7%95%D7%97_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%94_(%D7%94%D7%9E%D7%A1%D7%9C%D7%95%D7%9C_%D7%94%D7%90%D7%93%D7%95%D7%9D)" title="פיצויים לבעלי עסקים בישובי ספר בגין הפסד או מניעת רווח עקב מלחמה (המסלול האדום)">פיצוי על הפסדים לעסקים ביישובי ספר (המסלול האדום)</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%A9%D7%A9%D7%99%D7%A8%D7%AA%D7%95_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי לבעלי עסקים ששירתו במילואים במהלך מלחמת חרבות ברזל">פיצוי לבעלי עסקים ששירתו במילואים במהלך מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%9E%D7%A2%D7%A1%D7%99%D7%A7%D7%99%D7%9D_%D7%A9%D7%A9%D7%99%D7%9C%D7%9E%D7%95_%D7%A9%D7%9B%D7%A8_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D_%D7%95%D7%A0%D7%A2%D7%93%D7%A8%D7%95_%D7%9E%D7%94%D7%A2%D7%91%D7%95%D7%93%D7%94_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי למעסיקים ששילמו שכר לעובדים שפונו מבתיהם ונעדרו מהעבודה במלחמת חרבות ברזל">פיצוי למעסיקים ששילמו שכר לעובדים שפונו מבתיהם ונעדרו מהעבודה במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9C%D7%9E%D7%A2%D7%95%D7%A0%D7%95%D7%AA_%D7%95%D7%9E%D7%A9%D7%A4%D7%97%D7%AA%D7%95%D7%A0%D7%99%D7%9D_%D7%9E%D7%95%D7%9B%D7%A8%D7%99%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק למעונות ומשפחתונים מוכרים בתקופת מלחמת חרבות ברזל">מענק למעונות ומשפחתונים מוכרים בתקופת מלחמת חרבות ברזל</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/sa140124-1?utm_source=InforuMail&amp;utm_medium=email&amp;utm_campaign=%D7%94%D7%95%D7%93%D7%A2%D7%95%D7%AA+%D7%9C%D7%A2%D7%99%D7%AA%D7%95%D7%A0%D7%95%D7%AA%2C+%D7%94%D7%95%D7%93%D7%A2%D7%95%D7%AA+%D7%9C%D7%A6%D7%99%D7%91%D7%95%D7%A8%2C+%D7%9E%D7%A1+%D7%94%D7%9B%D7%A0%D7%A1%D7%94%2C+%D7%9E%D7%A1+%D7%A2%D7%A8%D7%9A+%D7%9E%D7%95%D7%A1%D7%A3">מתווה פיצויים ייעודי למגדלים בענף ההטלה (לולנים) ביישובי הספר</a></li>
<li><a href="/he/%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%9E%D7%A7%D7%93%D7%9E%D7%95%D7%AA_%D7%9C%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91%D7%99_%D7%A1%D7%A4%D7%A8_%D7%91%D7%93%D7%A8%D7%95%D7%9D_%D7%95%D7%91%D7%A6%D7%A4%D7%95%D7%9F_%D7%94%D7%90%D7%A8%D7%A5_%D7%91%D7%A9%D7%9C_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="תשלום מקדמות לבעלי עסקים ביישובי ספר בדרום ובצפון הארץ בשל מלחמת חרבות ברזל">תשלום מקדמות לבעלי עסקים ביישובי ספר בדרום ובצפון הארץ בשל מלחמת חרבות ברזל</a></li></ul></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%9E%D7%A2%D7%A1%D7%99%D7%A7%D7%99%D7%9D_%D7%A9%D7%A9%D7%99%D7%9C%D7%9E%D7%95_%D7%94%D7%A4%D7%A8%D7%A9%D7%95%D7%AA_%D7%A1%D7%95%D7%A6%D7%99%D7%90%D7%9C%D7%99%D7%95%D7%AA_%D7%A2%D7%91%D7%95%D7%A8_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%94%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%94%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי למעסיקים ששילמו הפרשות סוציאליות עבור עובדיהם בתקופת שירות המילואים במלחמת חרבות ברזל">פיצוי למעסיקים ששילמו הפרשות סוציאליות עבור עובדיהם בתקופת שירות המילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A4%D7%A7%D7%93%D7%95%D7%AA_%D7%9C%D7%91%D7%99%D7%98%D7%95%D7%97_%D7%A4%D7%A0%D7%A1%D7%99%D7%95%D7%A0%D7%99_%D7%9C%D7%A2%D7%A6%D7%9E%D7%90%D7%99%D7%9D_%D7%A9%D7%A0%D7%97%D7%98%D7%A4%D7%95_%D7%90%D7%95_%D7%94%D7%95%D7%9B%D7%A8%D7%96%D7%95_%D7%9B%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הפקדות לביטוח פנסיוני לעצמאים שנחטפו או הוכרזו כנעדרים במלחמת חרבות ברזל">הפקדות לביטוח פנסיוני לעצמאים שנחטפו או הוכרזו כנעדרים</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.sba.org.il/hb/Pages/HB_biz.aspx?utm_source=go.gov.il&amp;utm_medium=referral">פיצויים וכלים לעסקים בחירום</a> - באתר הסוכנות לעסקים קטנים ובינוניים</li>
<li><a href="/he/%D7%91%D7%99%D7%98%D7%95%D7%9C_%D7%94%D7%92%D7%91%D7%9C%D7%AA_%D7%97%D7%A9%D7%91%D7%95%D7%9F_%D7%91%D7%92%D7%99%D7%9F_%D7%A6%27%D7%A7_%D7%A9%D7%97%D7%96%D7%A8_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="ביטול הגבלת חשבון בגין צ&#39;ק שחזר בתקופת מלחמת חרבות ברזל">ביטול הגבלת חשבון בגין צ'ק שחזר בתקופת מלחמת חרבות ברזל</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.boi.org.il/bank-of-israel/iron-swords/boi-outline-banks/">הקלות בתחום הבנקאות והאשראי</a> באתר בנק ישראל</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/about/news/Pages/MarctMkovnet.aspx">מערכת מקוונת למעסיקים להגשת בקשות להחזרי מילואים</a> באתר הביטוח הלאומי</li>
<li><b><a href="/he/%D7%9E%D7%99%D7%93%D7%A2_%D7%9C%D7%A2%D7%9E%D7%95%D7%AA%D7%95%D7%AA_%D7%95%D7%9C%D7%97%D7%91%D7%A8%D7%95%D7%AA_%D7%9C%D7%AA%D7%95%D7%A2%D7%9C%D7%AA_%D7%94%D7%A6%D7%99%D7%91%D7%95%D7%A8_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מידע לעמותות ולחברות לתועלת הציבור בתקופת מלחמת חרבות ברזל">מידע לעמותות ולחברות לתועלת הציבור בתקופת מלחמת חרבות ברזל</a></b></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://translation.biu.ac.il/node/917">שירותי תרגום בעל-פה בחינם לארגונים חברתיים מטעם אונ' בר-אילן</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="מילואים"></span><span class="mw-headline" id=".D7.9E.D7.99.D7.9C.D7.95.D7.90.D7.99.D7.9D">מילואים</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" class="mw-redirect" title="מדריך למשרתי מילואים במלחמת חרבות ברזל">מדריך למשרתי מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%9E%D7%A9%D7%A8%D7%AA%D7%99%D7%9D_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות למי שמשרתים במילואים במהלך מלחמת חרבות ברזל">שאלות ותשובות למי שמשרתים במילואים</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%9E%D7%A9%D7%AA%D7%97%D7%A8%D7%A8%D7%99%D7%9D_%D7%9E%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך למשתחררים משירות מילואים במלחמת חרבות ברזל">מדריך למשתחררים משירות מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%95%D7%94%D7%97%D7%96%D7%A8%D7%99_%D7%94%D7%95%D7%A6%D7%90%D7%95%D7%AA_%D7%91%D7%92%D7%99%D7%9F_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="תשלומים והחזרי הוצאות בגין שירות מילואים">תשלומים והחזרי הוצאות בגין שירות מילואים</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%91%D7%A0%D7%A7%D7%99%D7%9D_%D7%95%D7%91%D7%97%D7%91%D7%A8%D7%95%D7%AA_%D7%94%D7%90%D7%A9%D7%A8%D7%90%D7%99_%D7%9C%D7%AA%D7%95%D7%A9%D7%91%D7%99_%D7%94%D7%93%D7%A8%D7%95%D7%9D,_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D,_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%94%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%95%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%94%D7%94%D7%A8%D7%95%D7%92%D7%99%D7%9D,_%D7%94%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%94%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" class="mw-redirect" title="הקלות בבנקים ובחברות האשראי לתושבי הדרום, למפונים מבתיהם, למשרתי המילואים ולמשפחות ההרוגים, החטופים והנעדרים במלחמת חרבות ברזל">הקלות בבנקים ובחברות האשראי</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%97%D7%95%D7%9D_%D7%94%D7%AA%D7%9B%D7%A0%D7%95%D7%9F_%D7%95%D7%94%D7%91%D7%A0%D7%99%D7%99%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%A8%D7%A9%D7%95%D7%AA_%D7%94%D7%9E%D7%99%D7%A1%D7%99%D7%9D_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ברשות המיסים למשרתי מילואים במלחמת חרבות ברזל">הקלות ברשות המיסים למשרתי מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלומים לרשויות שונות בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלומים לרשויות שונות בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית פעולה שיש לבצע לפי פסק דין בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית פעולה שיש לבצע לפי פסק דין בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלום או פעולה שיש לבצע לפי חוזה החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלומים לרשויות שונות בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלומים לרשויות שונות החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית פעולה שיש לבצע לפי פסק דין בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית פעולה שיש לבצע לפי פסק דין החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%90%D7%95_%D7%91%D7%AA%D7%95%D7%9D_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="איסור פיטורי עובד במהלך או בתום שירות מילואים">איסור פיטורי עובד במהלך או בתום שירות מילואים</a></li>
<li><a href="/he/%D7%94%D7%99%D7%A2%D7%93%D7%A8%D7%95%D7%AA_%D7%9E%D7%A2%D7%91%D7%95%D7%93%D7%94_%D7%A9%D7%9C_%D7%91%D7%A0%D7%99_%D7%96%D7%95%D7%92_%D7%90%D7%95_%D7%A9%D7%95%D7%AA%D7%A4%D7%99%D7%9D_%D7%9C%D7%94%D7%95%D7%A8%D7%95%D7%AA_%D7%A9%D7%9C_%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="היעדרות מעבודה של בני זוג או שותפים להורות של משרתי מילואים במלחמת חרבות ברזל">היעדרות מעבודה של בני זוג או שותפים להורות של משרתי מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A0%D7%A2%D7%93%D7%A8%D7%95_%D7%9E%D7%A2%D7%91%D7%95%D7%93%D7%AA%D7%9D_%D7%9C%D7%A6%D7%95%D7%A8%D7%9A_%D7%94%D7%A9%D7%92%D7%97%D7%94_%D7%A2%D7%9C_%D7%99%D7%9C%D7%93%D7%99%D7%94%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="איסור פיטורי עובדים שנעדרו מעבודתם לצורך השגחה על ילדיהם במהלך מלחמת חרבות ברזל">איסור פיטורי בני זוג של משרתי מילואים או שותפיהם להורות שנעדרו מעבודתם לצורך השגחה על ילדיהם</a></li>
<li><a href="/he/%D7%A6%D7%91%D7%99%D7%A8%D7%AA_%D7%99%D7%9E%D7%99_%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9C%D7%90_%D7%99%D7%9B%D7%9C%D7%95_%D7%9C%D7%A0%D7%A6%D7%9C_%D7%90%D7%AA_%D7%99%D7%9E%D7%99_%D7%94%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל">צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%AA%D7%A0%D7%90%D7%99_%D7%94%D7%94%D7%AA%D7%9E%D7%97%D7%95%D7%AA_%D7%95%D7%91%D7%9E%D7%91%D7%97%D7%A0%D7%99_%D7%94%D7%94%D7%A1%D7%9E%D7%9B%D7%94_%D7%9E%D7%98%D7%A2%D7%9D_%D7%9E%D7%A9%D7%A8%D7%93_%D7%94%D7%9E%D7%A9%D7%A4%D7%98%D7%99%D7%9D_%D7%91%D7%A2%D7%A7%D7%91%D7%95%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות בתנאי ההתמחות ובמבחני ההסמכה מטעם משרד המשפטים בעקבות מלחמת חרבות ברזל">הקלות בתנאי ההתמחות ובמבחני ההסמכה מטעם משרד המשפטים</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%94%D7%95%D7%A6%D7%90%D7%94_%D7%9C%D7%A4%D7%95%D7%A2%D7%9C_%D7%95%D7%91%D7%9E%D7%A8%D7%9B%D7%96_%D7%9C%D7%92%D7%91%D7%99%D7%99%D7%AA_%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות עקב מלחמת חרבות ברזל">הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות</a></li>
<li><a href="/he/%D7%94%D7%A4%D7%A8%D7%A9%D7%95%D7%AA_%D7%9C%D7%91%D7%99%D7%98%D7%95%D7%97_%D7%A4%D7%A0%D7%A1%D7%99%D7%95%D7%A0%D7%99_%D7%95%D7%9C%D7%A7%D7%A8%D7%9F_%D7%94%D7%A9%D7%AA%D7%9C%D7%9E%D7%95%D7%AA_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="הפרשות לביטוח פנסיוני ולקרן השתלמות לעובדים במילואים">הפרשות לביטוח פנסיוני ולקרן השתלמות לעובדים במילואים</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%9E%D7%A2%D7%A1%D7%99%D7%A7%D7%99%D7%9D_%D7%A9%D7%A9%D7%99%D7%9C%D7%9E%D7%95_%D7%94%D7%A4%D7%A8%D7%A9%D7%95%D7%AA_%D7%A1%D7%95%D7%A6%D7%99%D7%90%D7%9C%D7%99%D7%95%D7%AA_%D7%A2%D7%91%D7%95%D7%A8_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%94%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%94%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי למעסיקים ששילמו הפרשות סוציאליות עבור עובדיהם בתקופת שירות המילואים במלחמת חרבות ברזל">פיצוי למעסיקים ששילמו הפרשות סוציאליות עבור עובדיהם בתקופת שירות המילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%9C%D7%97%D7%99%D7%99%D7%9C%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%A9%D7%94%D7%9D_%D7%94%D7%95%D7%A8%D7%99%D7%9D_%D7%99%D7%97%D7%99%D7%93%D7%99%D7%9D" class="mw-redirect" title="הקלות לחיילי מילואים שהם הורים יחידים">הקלות לחיילי מילואים שהם הורים יחידים</a></li>
<li><a href="/he/%D7%92%D7%99%D7%95%D7%A1_%D7%9B%D7%9C%D7%99_%D7%A8%D7%9B%D7%91_%D7%A4%D7%A8%D7%98%D7%99%D7%99%D7%9D,_%D7%A6%D7%99%D7%95%D7%93_%D7%94%D7%A0%D7%93%D7%A1%D7%99_%D7%95%D7%9E%D7%AA%D7%A7%D7%A0%D7%99%D7%9D_%D7%90%D7%96%D7%A8%D7%97%D7%99%D7%99%D7%9D_%D7%91%D7%A9%D7%A2%D7%AA_%D7%97%D7%99%D7%A8%D7%95%D7%9D" title="גיוס כלי רכב פרטיים, ציוד הנדסי ומתקנים אזרחיים בשעת חירום">גיוס כלי רכב פרטיים, ציוד הנדסי ומתקנים אזרחיים בשעת חירום</a></li>
<li><a href="/he/%D7%A7%D7%A8%D7%99%D7%90%D7%94_%D7%9C%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%A6%D7%95_%D7%97%D7%A8%D7%99%D7%92" title="קריאה למילואים בצו חריג">קריאה למילואים בצו חריג</a></li>
<li><a href="/he/%D7%A7%D7%A8%D7%99%D7%90%D7%94_%D7%9C%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%A6%D7%95_%D7%97%D7%99%D7%A8%D7%95%D7%9D_(%D7%A6%D7%95_8)" title="קריאה למילואים בצו חירום (צו 8)">קריאה למילואים בצו חירום (צו 8)</a></li>
<li><a href="/he/%D7%A7%D7%A8%D7%99%D7%90%D7%94_%D7%9C%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%A6%D7%95_%D7%9E%D7%99%D7%95%D7%97%D7%93_(%D7%A6%D7%95_9)" title="קריאה למילואים בצו מיוחד (צו 9)">קריאה למילואים בצו מיוחד (צו 9)</a></li>
<li><a href="/he/%D7%94%D7%AA%D7%A0%D7%93%D7%91%D7%95%D7%AA_%D7%9C%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%A9%D7%A2%D7%AA_%D7%97%D7%99%D7%A8%D7%95%D7%9D" title="התנדבות לשירות מילואים בשעת חירום">התנדבות לשירות מילואים בשעת חירום</a></li>
<li><a href="/he/%D7%94%D7%92%D7%A9%D7%AA_%D7%91%D7%A7%D7%A9%D7%94_%D7%9C%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%93%D7%99%D7%95%D7%9F_%D7%91%D7%91%D7%99%D7%AA_%D7%9E%D7%A9%D7%A4%D7%98" title="הגשת בקשה לדחיית דיון בבית משפט">דחיית דיון בבית משפט בעת מילואים</a></li>
<li><b><a href="/he/%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="מילואים">מילואים</a></b></li>
<li><b><a href="/he/%D7%A2%D7%91%D7%95%D7%93%D7%94_%D7%95%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="עבודה ומילואים">עבודה ומילואים</a></b></li>
<li><b><a href="/he/%D7%9E%D7%A9%D7%A4%D7%97%D7%94_%D7%95%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="משפחה ומילואים">משפחה ומילואים</a></b></li>
<li><b><a href="/he/%D7%A1%D7%98%D7%95%D7%93%D7%A0%D7%98%D7%99%D7%9D_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="סטודנטים במילואים">סטודנטים במילואים</a></b></li>
<li><a href="/he/%D7%A1%D7%98%D7%95%D7%93%D7%A0%D7%98%D7%99%D7%95%D7%AA" title="סטודנטיות">סטודנטיות בנות זוג של מילואימניקים</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="מענקים_ותשלומים_למשרתי_מילואים"></span><span class="mw-headline" id=".D7.9E.D7.A2.D7.A0.D7.A7.D7.99.D7.9D_.D7.95.D7.AA.D7.A9.D7.9C.D7.95.D7.9E.D7.99.D7.9D_.D7.9C.D7.9E.D7.A9.D7.A8.D7.AA.D7.99_.D7.9E.D7.99.D7.9C.D7.95.D7.90.D7.99.D7.9D">מענקים ותשלומים למשרתי מילואים</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%95%D7%94%D7%97%D7%96%D7%A8%D7%99_%D7%94%D7%95%D7%A6%D7%90%D7%95%D7%AA_%D7%91%D7%92%D7%99%D7%9F_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="תשלומים והחזרי הוצאות בגין שירות מילואים">תשלומים והחזרי הוצאות בגין שירות מילואים</a></li>
<li><a href="/he/%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%A2%D7%91%D7%95%D7%A8_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D" title="תשלום עבור שירות מילואים">תשלום עבור שירות מילואים</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9B%D7%9C%D7%9B%D7%9C%D7%AA_%D7%94%D7%91%D7%99%D7%AA_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%A4%D7%A2%D7%99%D7%9C%D7%99%D7%9D" title="מענק כלכלת הבית למשרתי מילואים פעילים">מענק כלכלת הבית למשרתי מילואים פעילים</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%94%D7%95%D7%A6%D7%90%D7%95%D7%AA_%D7%90%D7%99%D7%A9%D7%99%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A9%D7%99%D7%A8%D7%AA%D7%95_%D7%91%D7%9E%D7%A1%D7%92%D7%A8%D7%AA_%D7%A6%D7%95_8_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק הוצאות אישיות למי ששירתו במסגרת צו 8 במלחמת חרבות ברזל">מענק הוצאות אישיות למשרתים בצו 8</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%94%D7%95%D7%A6%D7%90%D7%95%D7%AA_%D7%90%D7%99%D7%A9%D7%99%D7%95%D7%AA_%D7%9E%D7%95%D7%92%D7%93%D7%9C_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A9%D7%99%D7%A8%D7%AA%D7%95_%D7%91%D7%A6%D7%95_8_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק הוצאות אישיות מוגדל למי ששירתו בצו 8 במלחמת חרבות ברזל">מענק הוצאות אישיות מוגדל למי ששירתו בצו 8 במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9E%D7%A9%D7%A4%D7%97%D7%94_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A9%D7%99%D7%A8%D7%AA%D7%95_%D7%91%D7%9E%D7%A1%D7%92%D7%A8%D7%AA_%D7%A6%D7%95_8_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק משפחה למי ששירתו במסגרת צו 8 במלחמת חרבות ברזל">מענק משפחה להורים לילדים עד גיל 14 שמשרתים בצו 8</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9E%D7%A9%D7%A4%D7%97%D7%94_%D7%9E%D7%95%D7%92%D7%93%D7%9C_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A9%D7%99%D7%A8%D7%AA%D7%95_%D7%91%D7%A6%D7%95_8_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק משפחה מוגדל למי ששירתו בצו 8 במלחמת חרבות ברזל">מענק משפחה מוגדל למי ששירתו בצו 8 במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%A9%D7%94%D7%95%D7%90_%D7%94%D7%95%D7%A8%D7%94_%D7%9C%D7%99%D7%9C%D7%93_%D7%A2%D7%9D_%D7%A6%D7%A8%D7%9B%D7%99%D7%9D_%D7%9E%D7%99%D7%95%D7%97%D7%93%D7%99%D7%9D" title="מענק למשרת מילואים במלחמת חרבות ברזל שהוא הורה לילד עם צרכים מיוחדים">מענק למשרת מילואים במלחמת חרבות ברזל שהוא הורה לילד עם צרכים מיוחדים</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%9C%D7%A2%D7%95%D7%91%D7%93_%D7%A9%D7%94%D7%95%D7%A6%D7%90_%D7%9C%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%9C%D7%90_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%A9%D7%9C_%D7%91%D7%9F_%D7%96%D7%95%D7%92%D7%95_%D7%90%D7%95_%D7%94%D7%95%D7%A8%D7%94_%D7%9E%D7%A9%D7%95%D7%AA%D7%A3_%D7%9C%D7%99%D7%9C%D7%93%D7%95_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק לעובד שהוצא לחופשה ללא תשלום בתקופת מילואים של בן זוגו או הורה משותף לילדו במלחמת חרבות ברזל">מענק לעובד שהוצא לחופשה ללא תשלום בתקופת מילואים של בן זוגו או הורה משותף לילד</a></li>
<li><a href="/he/%D7%A1%D7%99%D7%95%D7%A2_%D7%9B%D7%A1%D7%A4%D7%99_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99%D7%9D_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%A9%D7%9E%D7%A8%D7%9B%D7%96_%D7%97%D7%99%D7%99%D7%94%D7%9D_%D7%9C%D7%90_%D7%91%D7%99%D7%A9%D7%A8%D7%90%D7%9C" title="סיוע כספי למשרתים במילואים במלחמת חרבות ברזל שמרכז חייהם לא בישראל">סיוע כספי למשרתים במילואים במלחמת חרבות ברזל שמרכז חייהם לא בישראל</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%91%D7%92%D7%99%D7%9F_%D7%94%D7%95%D7%A6%D7%90%D7%95%D7%AA_%D7%90%D7%99%D7%A9%D7%99%D7%95%D7%AA_%D7%97%D7%A8%D7%99%D7%92%D7%95%D7%AA_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי בגין הוצאות אישיות חריגות למשרתי מילואים במלחמת חרבות ברזל">פיצוי בגין הוצאות אישיות חריגות למשרתי מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%A1%D7%98%D7%95%D7%93%D7%A0%D7%98_%D7%91%D7%92%D7%99%D7%9F_%D7%94%D7%95%D7%A6%D7%90%D7%95%D7%AA_%D7%A9%D7%A0%D7%92%D7%A8%D7%9E%D7%95_%D7%91%D7%A2%D7%A7%D7%91%D7%95%D7%AA_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי לסטודנט בגין הוצאות שנגרמו בעקבות שירות מילואים במלחמת חרבות ברזל">פיצוי לסטודנט בגין הוצאות שנגרמו בעקבות שירות מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%A2%D7%91%D7%95%D7%A8_%D7%98%D7%99%D7%A1%D7%95%D7%AA_%D7%95%D7%91%D7%99%D7%98%D7%95%D7%9C%D7%99_%D7%97%D7%95%D7%A4%D7%A9%D7%95%D7%AA_%D7%9C%D7%97%D7%99%D7%99%D7%9C%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%A9%D7%92%D7%95%D7%99%D7%A1%D7%95_%D7%91%D7%A6%D7%95_8_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי עבור טיסות וביטולי חופשות לחיילי מילואים שגויסו בצו 8 במלחמת חרבות ברזל">פיצוי עבור טיסות וביטולי חופשות לחיילי מילואים שגויסו בצו 8 במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%91%D7%92%D7%99%D7%9F_%D7%90%D7%95%D7%91%D7%93%D7%9F_%D7%94%D7%9B%D7%A0%D7%A1%D7%94_%D7%9C%D7%91%D7%A0%D7%99_%D7%9E%D7%A9%D7%A4%D7%97%D7%94_%D7%A9%D7%9C_%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי בגין אובדן הכנסה לבני משפחה של משרתי מילואים במלחמת חרבות ברזל">פיצוי בגין אובדן הכנסה לבני משפחה של משרתי מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%97%D7%96%D7%A8_%D7%97%D7%93_%D7%A4%D7%A2%D7%9E%D7%99_%D7%A2%D7%91%D7%95%D7%A8_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%9C%D7%98%D7%99%D7%A4%D7%95%D7%9C_%D7%91%D7%AA%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%99%D7%AA_%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%AA%D7%95_%D7%A9%D7%9C_%D7%9E%D7%A9%D7%A8%D7%AA_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="החזר חד פעמי עבור תשלום לטיפול בתקלות בית למשפחתו של משרת במילואים במלחמת חרבות ברזל">החזר חד פעמי עבור תשלום לטיפול בתקלות בית למשפחתו של משרת במילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A1%D7%99%D7%95%D7%A2_%D7%91%D7%9E%D7%99%D7%9E%D7%95%D7%9F_%D7%98%D7%99%D7%A4%D7%95%D7%9C_%D7%A8%D7%92%D7%A9%D7%99,_%D7%98%D7%99%D7%A4%D7%95%D7%9C_%D7%A0%D7%A4%D7%A9%D7%99_%D7%95%D7%98%D7%99%D7%A4%D7%95%D7%9C_%D7%9E%D7%A9%D7%9C%D7%99%D7%9D_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="סיוע במימון טיפול רגשי, טיפול נפשי וטיפול משלים למשרתי מילואים במלחמת חרבות ברזל">סיוע במימון טיפול רגשי, טיפול נפשי וטיפול משלים למשרתי מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A1%D7%99%D7%95%D7%A2_%D7%91%D7%9E%D7%99%D7%9E%D7%95%D7%9F_%D7%98%D7%99%D7%A4%D7%95%D7%9C_%D7%96%D7%95%D7%92%D7%99_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="סיוע במימון טיפול זוגי למשרתי מילואים במלחמת חרבות ברזל">סיוע במימון טיפול זוגי למשרתי מילואים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%9C%D7%91%D7%A2%D7%9C%D7%99_%D7%A2%D7%A1%D7%A7%D7%99%D7%9D_%D7%A9%D7%A9%D7%99%D7%A8%D7%AA%D7%95_%D7%91%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי לבעלי עסקים ששירתו במילואים במהלך מלחמת חרבות ברזל">פיצוי לבעלי עסקים ששירתו במילואים במהלך מלחמת חרבות ברזל</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="חטופים,_נעדרים_ובני_משפחותיהם"></span><span class="mw-headline" id=".D7.97.D7.98.D7.95.D7.A4.D7.99.D7.9D.2C_.D7.A0.D7.A2.D7.93.D7.A8.D7.99.D7.9D_.D7.95.D7.91.D7.A0.D7.99_.D7.9E.D7.A9.D7.A4.D7.97.D7.95.D7.AA.D7.99.D7.94.D7.9D">חטופים, נעדרים ובני משפחותיהם</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%96%D7%9B%D7%95%D7%AA%D7%95%D7%9F_%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%A9%D7%9C_%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="זכותון למשפחות של חטופים ונעדרים במלחמת חרבות ברזל">זכותון למשפחות של חטופים ונעדרים במלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7%D7%99%D7%9D_%D7%9C%D7%9E%D7%99%D7%9E%D7%95%D7%9F_%D7%94%D7%95%D7%A6%D7%90%D7%95%D7%AA_%D7%9C%D7%91%D7%A0%D7%99_%D7%9E%D7%A9%D7%A4%D7%97%D7%94_%D7%A9%D7%9C_%D7%A9%D7%91%D7%95%D7%99%D7%99%D7%9D,_%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D" title="מענקים למימון הוצאות לבני משפחה של שבויים, חטופים ונעדרים">מענקים למימון הוצאות לבני משפחה של שבויים, חטופים ונעדרים</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%91%D7%A0%D7%A7%D7%99%D7%9D_%D7%95%D7%91%D7%97%D7%91%D7%A8%D7%95%D7%AA_%D7%94%D7%90%D7%A9%D7%A8%D7%90%D7%99_%D7%9C%D7%AA%D7%95%D7%A9%D7%91%D7%99_%D7%94%D7%93%D7%A8%D7%95%D7%9D,_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D,_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%94%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%95%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%94%D7%94%D7%A8%D7%95%D7%92%D7%99%D7%9D,_%D7%94%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%94%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" class="mw-redirect" title="הקלות בבנקים ובחברות האשראי לתושבי הדרום, למפונים מבתיהם, למשרתי המילואים ולמשפחות ההרוגים, החטופים והנעדרים במלחמת חרבות ברזל">הקלות בבנקים ובחברות האשראי</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%97%D7%95%D7%9D_%D7%94%D7%AA%D7%9B%D7%A0%D7%95%D7%9F_%D7%95%D7%94%D7%91%D7%A0%D7%99%D7%99%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%94%D7%95%D7%A6%D7%90%D7%94_%D7%9C%D7%A4%D7%95%D7%A2%D7%9C_%D7%95%D7%91%D7%9E%D7%A8%D7%9B%D7%96_%D7%9C%D7%92%D7%91%D7%99%D7%99%D7%AA_%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות עקב מלחמת חרבות ברזל">הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלומים לרשויות שונות בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלומים לרשויות שונות בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית פעולה שיש לבצע לפי פסק דין בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית פעולה שיש לבצע לפי פסק דין בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלום או פעולה שיש לבצע לפי חוזה החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלומים לרשויות שונות בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלומים לרשויות שונות החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית פעולה שיש לבצע לפי פסק דין בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית פעולה שיש לבצע לפי פסק דין החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%A4%D7%98%D7%95%D7%A8_%D7%9E%D7%90%D7%A8%D7%A0%D7%95%D7%A0%D7%94_%D7%9C%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D" title="פטור מארנונה לחטופים ונעדרים">פטור מארנונה לחטופים ונעדרים</a></li>
<li><a href="/he/%D7%93%D7%99%D7%95%D7%A8_%D7%A6%D7%99%D7%91%D7%95%D7%A8%D7%99_%D7%9C%D7%93%D7%99%D7%99%D7%A8%D7%99%D7%9D_%D7%9E%D7%9E%D7%A9%D7%99%D7%9B%D7%99%D7%9D_%D7%A9%D7%91%D7%9F_%D7%9E%D7%A9%D7%A4%D7%97%D7%AA%D7%9D_%D7%A0%D7%94%D7%A8%D7%92,_%D7%A0%D7%97%D7%98%D7%A3_%D7%90%D7%95_%D7%A0%D7%A2%D7%93%D7%A8_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="דיור ציבורי לדיירים ממשיכים שבן משפחתם נהרג, נחטף או נעדר בתקופת מלחמת חרבות ברזל">דיור ציבורי לדיירים ממשיכים שבן משפחתם נחטף או נעדר</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%94%D7%9D_%D7%90%D7%95_%D7%91%D7%A0%D7%99_%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D_%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%90%D7%95_%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="איסור פיטורי עובדים שהם או בני משפחותיהם חטופים או נעדרים במהלך מלחמת חרבות ברזל">איסור פיטורי עובדים שהם או בני משפחותיהם חטופים או נעדרים</a></li>
<li><a href="/he/%D7%94%D7%A4%D7%A7%D7%93%D7%95%D7%AA_%D7%9C%D7%91%D7%99%D7%98%D7%95%D7%97_%D7%A4%D7%A0%D7%A1%D7%99%D7%95%D7%A0%D7%99_%D7%9C%D7%A2%D7%A6%D7%9E%D7%90%D7%99%D7%9D_%D7%A9%D7%A0%D7%97%D7%98%D7%A4%D7%95_%D7%90%D7%95_%D7%94%D7%95%D7%9B%D7%A8%D7%96%D7%95_%D7%9B%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הפקדות לביטוח פנסיוני לעצמאים שנחטפו או הוכרזו כנעדרים במלחמת חרבות ברזל">הפקדות לביטוח פנסיוני לעצמאים שנחטפו או הוכרזו כנעדרים</a></li>
<li><a href="/he/%D7%94%D7%A4%D7%A7%D7%93%D7%95%D7%AA_%D7%9C%D7%91%D7%99%D7%98%D7%95%D7%97_%D7%A4%D7%A0%D7%A1%D7%99%D7%95%D7%A0%D7%99_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A0%D7%97%D7%98%D7%A4%D7%95_%D7%90%D7%95_%D7%94%D7%95%D7%9B%D7%A8%D7%96%D7%95_%D7%9B%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הפקדות לביטוח פנסיוני לעובדים שנחטפו או הוכרזו כנעדרים במלחמת חרבות ברזל">הפקדות לביטוח פנסיוני לעובדים שנחטפו או הוכרזו כנעדרים</a></li>
<li><a href="/he/%D7%A7%D7%A6%D7%91%D7%94_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A0%D7%A4%D7%9C%D7%95_%D7%91%D7%A9%D7%91%D7%99_%D7%9B%D7%90%D7%A9%D7%A8_%D7%A9%D7%99%D7%A8%D7%AA%D7%95_%D7%91%D7%A6%D7%94%22%D7%9C_%D7%90%D7%95_%D7%91%D7%9B%D7%95%D7%97%D7%95%D7%AA_%D7%94%D7%91%D7%99%D7%98%D7%97%D7%95%D7%9F_%D7%95%D7%9C%D7%9E%D7%99_%D7%A9%D7%A0%D7%97%D7%98%D7%A4%D7%95_%D7%91%D7%A4%D7%A2%D7%95%D7%9C%D7%AA_%D7%90%D7%99%D7%91%D7%94" class="mw-redirect" title="קצבה למי שנפלו בשבי כאשר שירתו בצה&quot;ל או בכוחות הביטחון ולמי שנחטפו בפעולת איבה">קצבה למי שנחטפו לאחר שחרורם</a></li>
<li><a href="/he/%D7%A6%D7%91%D7%99%D7%A8%D7%AA_%D7%99%D7%9E%D7%99_%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9C%D7%90_%D7%99%D7%9B%D7%9C%D7%95_%D7%9C%D7%A0%D7%A6%D7%9C_%D7%90%D7%AA_%D7%99%D7%9E%D7%99_%D7%94%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל">צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.kivunimrights.com/">סיוע במימוש זכויות של מי שנפגעו מהמלחמה או מפעולות האיבה</a> מוקד ערב"ה בשיתוף מערך הדיגיטל הלאומי</li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="משפחות_שכולות"></span><span class="mw-headline" id=".D7.9E.D7.A9.D7.A4.D7.97.D7.95.D7.AA_.D7.A9.D7.9B.D7.95.D7.9C.D7.95.D7.AA">משפחות שכולות</span></h2></div>
	<div class="portal-box-inner">
<ul><li><b><a href="/he/%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%A9%D7%9B%D7%95%D7%9C%D7%95%D7%AA_%D7%A9%D7%9C_%D7%97%D7%9C%D7%9C%D7%99_%D7%A4%D7%A2%D7%95%D7%9C%D7%95%D7%AA_%D7%90%D7%99%D7%91%D7%94" title="משפחות שכולות של חללי פעולות איבה">משפחות שכולות של חללי פעולות איבה</a></b></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%91%D7%A0%D7%A7%D7%99%D7%9D_%D7%95%D7%91%D7%97%D7%91%D7%A8%D7%95%D7%AA_%D7%94%D7%90%D7%A9%D7%A8%D7%90%D7%99_%D7%9C%D7%AA%D7%95%D7%A9%D7%91%D7%99_%D7%94%D7%93%D7%A8%D7%95%D7%9D,_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D,_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%94%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%95%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%94%D7%94%D7%A8%D7%95%D7%92%D7%99%D7%9D,_%D7%94%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%94%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" class="mw-redirect" title="הקלות בבנקים ובחברות האשראי לתושבי הדרום, למפונים מבתיהם, למשרתי המילואים ולמשפחות ההרוגים, החטופים והנעדרים במלחמת חרבות ברזל">הקלות בבנקים ובחברות האשראי</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%97%D7%95%D7%9D_%D7%94%D7%AA%D7%9B%D7%A0%D7%95%D7%9F_%D7%95%D7%94%D7%91%D7%A0%D7%99%D7%99%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%94%D7%95%D7%A6%D7%90%D7%94_%D7%9C%D7%A4%D7%95%D7%A2%D7%9C_%D7%95%D7%91%D7%9E%D7%A8%D7%9B%D7%96_%D7%9C%D7%92%D7%91%D7%99%D7%99%D7%AA_%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות עקב מלחמת חרבות ברזל">הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A7%D7%A8%D7%95%D7%91%D7%99%D7%94%D7%9D_%D7%A0%D7%94%D7%A8%D7%92%D7%95_%D7%91%D7%90%D7%99%D7%A8%D7%95%D7%A2%D7%99_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות למי שקרוביהם נהרגו באירועי מלחמת חרבות ברזל">שאלות ותשובות למי שקרוביהם נהרגו</a></li>
<li><a href="/he/%D7%93%D7%99%D7%95%D7%A8_%D7%A6%D7%99%D7%91%D7%95%D7%A8%D7%99_%D7%9C%D7%93%D7%99%D7%99%D7%A8%D7%99%D7%9D_%D7%9E%D7%9E%D7%A9%D7%99%D7%9B%D7%99%D7%9D_%D7%A9%D7%91%D7%9F_%D7%9E%D7%A9%D7%A4%D7%97%D7%AA%D7%9D_%D7%A0%D7%94%D7%A8%D7%92,_%D7%A0%D7%97%D7%98%D7%A3_%D7%90%D7%95_%D7%A0%D7%A2%D7%93%D7%A8_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="דיור ציבורי לדיירים ממשיכים שבן משפחתם נהרג, נחטף או נעדר בתקופת מלחמת חרבות ברזל">דיור ציבורי לדיירים ממשיכים שבן משפחתם נהרג</a></li>
<li><a href="/he/%D7%A7%D7%91%D7%95%D7%A8%D7%94_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="קבורה במהלך מלחמת חרבות ברזל">קבורה במהלך מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%95%D7%99%D7%AA%D7%95%D7%A8_%D7%A2%D7%9C_%D7%99%D7%A8%D7%95%D7%A9%D7%94_(%D7%94%D7%A1%D7%AA%D7%9C%D7%A7%D7%95%D7%AA_%D7%99%D7%95%D7%A8%D7%A9)_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="ויתור על ירושה (הסתלקות יורש) בתקופת מלחמת חרבות ברזל">ויתור על ירושה (הסתלקות יורש) בתקופת מלחמת חרבות ברזל</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.kivunimrights.com/">סיוע במימוש זכויות של מי שנפגעו מהמלחמה או מפעולות האיבה</a> מוקד ערב"ה בשיתוף מערך הדיגיטל הלאומי</li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="פגיעות_ברכוש"></span><span class="mw-headline" id=".D7.A4.D7.92.D7.99.D7.A2.D7.95.D7.AA_.D7.91.D7.A8.D7.9B.D7.95.D7.A9">פגיעות ברכוש</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%9E%D7%99_%D7%A9%D7%A8%D7%9B%D7%95%D7%A9%D7%9D_%D7%A0%D7%A4%D7%92%D7%A2_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="שאלות ותשובות למי שרכושם נפגע במהלך מלחמת חרבות ברזל">שאלות ותשובות למי שרכושם נפגע במלחמה</a></li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%A2%D7%9C_%D7%A0%D7%96%D7%A7_%D7%9C%D7%A8%D7%9B%D7%91_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91%D7%99_%D7%A2%D7%95%D7%98%D7%A3_%D7%A2%D7%96%D7%94_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="פיצוי על נזק לרכב ביישובי עוטף עזה עקב מלחמת חרבות ברזל">פיצוי על נזק לרכב ביישובי עוטף עזה</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/Departments/Guides/guide-for-civilians-property-damaged-in-hostilities?chapterIndex=1">מדריך לאזרחים שרכושם נפגע בפעולות מלחמה</a> באתר רשות המסים</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/apps/taxes/taxes/#/shelot-vetshuvot/he?CodNossePopUp=26,-1">שאלות ותשובות לנושאי נזק ישיר</a> באתר רשות המסים</li>
<li><a href="/he/%D7%A4%D7%99%D7%A6%D7%95%D7%99_%D7%A2%D7%9C_%D7%A0%D7%96%D7%A7_%D7%A9%D7%A0%D7%92%D7%A8%D7%9D_%D7%9C%D7%A8%D7%9B%D7%95%D7%A9_%D7%91%D7%92%D7%99%D7%9F_%D7%A4%D7%92%D7%99%D7%A2%D7%AA_%D7%90%D7%99%D7%91%D7%94_%D7%A2%D7%9C_%D7%A8%D7%A7%D7%A2_%D7%92%D7%96%D7%A2%D7%A0%D7%99_%D7%90%D7%95_%D7%9C%D7%90%D7%95%D7%9E%D7%A0%D7%99" title="פיצוי על נזק שנגרם לרכוש בגין פגיעת איבה על רקע גזעני או לאומני">פיצוי על נזק שנגרם לרכוש בגין פגיעת איבה על רקע גזעני או לאומני</a></li>
<li><a href="/he/%D7%91%D7%99%D7%98%D7%95%D7%97_%D7%A8%D7%A9%D7%95%D7%AA_%D7%A9%D7%9C_%D7%97%D7%A4%D7%A6%D7%99%D7%9D_%D7%91%D7%99%D7%AA%D7%99%D7%99%D7%9D_%D7%91%D7%A4%D7%A0%D7%99_%D7%A0%D7%96%D7%A7%D7%99_%D7%9E%D7%9C%D7%97%D7%9E%D7%94_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%95%D7%AA_%D7%90%D7%99%D7%91%D7%94" title="ביטוח רשות של חפצים ביתיים בפני נזקי מלחמה או פעולות איבה">ביטוח רשות של חפצים ביתיים בפני נזקי מלחמה או פעולות איבה</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.misim.gov.il/smpapainternet/frmOpen.aspx#">הגשת תביעה מקוונת בגין נזק ישיר</a></li>
<li><a href="/he/%D7%96%D7%9B%D7%95%D7%AA%D7%95%D7%9F_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%AA%D7%A7%D7%99%D7%A4%D7%95%D7%AA_%D7%92%D7%96%D7%A2%D7%A0%D7%99%D7%95%D7%AA_%D7%95%D7%9C%D7%90%D7%95%D7%9E%D7%A0%D7%99%D7%95%D7%AA" title="זכותון לנפגעי תקיפות גזעניות ולאומניות">זכותון לנפגעי תקיפות גזעניות ולאומניות</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.kivunimrights.com/">סיוע במימוש זכויות של מי שנפגעו מהמלחמה או מפעולות האיבה</a> מוקד ערב"ה בשיתוף מערך הדיגיטל הלאומי</li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="חיילים_ואנשי_כוחות_הביטחון_וההצלה"></span><span class="mw-headline" id=".D7.97.D7.99.D7.99.D7.9C.D7.99.D7.9D_.D7.95.D7.90.D7.A0.D7.A9.D7.99_.D7.9B.D7.95.D7.97.D7.95.D7.AA_.D7.94.D7.91.D7.99.D7.98.D7.97.D7.95.D7.9F_.D7.95.D7.94.D7.94.D7.A6.D7.9C.D7.94">חיילים ואנשי כוחות הביטחון וההצלה</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%97%D7%95%D7%9D_%D7%94%D7%AA%D7%9B%D7%A0%D7%95%D7%9F_%D7%95%D7%94%D7%91%D7%A0%D7%99%D7%99%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%94%D7%95%D7%A6%D7%90%D7%94_%D7%9C%D7%A4%D7%95%D7%A2%D7%9C_%D7%95%D7%91%D7%9E%D7%A8%D7%9B%D7%96_%D7%9C%D7%92%D7%91%D7%99%D7%99%D7%AA_%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות עקב מלחמת חרבות ברזל">הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית תשלומים לרשויות שונות בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית תשלומים לרשויות שונות בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%96%D7%9E%D7%9F_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%94_%D7%A9%D7%91%D7%99%D7%9F_07.10.2023_%D7%9C-31.12.2023" title="דחיית פעולה שיש לבצע לפי פסק דין בזמן מלחמת חרבות ברזל בתקופה שבין 07.10.2023 ל-31.12.2023">דחיית פעולה שיש לבצע לפי פסק דין בתקופה שבין 07.10.2023 ל-31.12.2023</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%95_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%96%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלום או פעולה שיש לבצע לפי חוזה בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלום או פעולה שיש לבצע לפי חוזה החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%AA%D7%A9%D7%9C%D7%95%D7%9E%D7%99%D7%9D_%D7%9C%D7%A8%D7%A9%D7%95%D7%99%D7%95%D7%AA_%D7%A9%D7%95%D7%A0%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית תשלומים לרשויות שונות בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית תשלומים לרשויות שונות החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%93%D7%97%D7%99%D7%99%D7%AA_%D7%A4%D7%A2%D7%95%D7%9C%D7%94_%D7%A9%D7%99%D7%A9_%D7%9C%D7%91%D7%A6%D7%A2_%D7%9C%D7%A4%D7%99_%D7%A4%D7%A1%D7%A7_%D7%93%D7%99%D7%9F_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C_%D7%94%D7%97%D7%9C_%D7%9E-01.01.2024" title="דחיית פעולה שיש לבצע לפי פסק דין בתקופת מלחמת חרבות ברזל החל מ-01.01.2024">דחיית פעולה שיש לבצע לפי פסק דין החל מ-01.01.2024</a></li>
<li><a href="/he/%D7%94%D7%AA%D7%A0%D7%93%D7%91%D7%95%D7%AA_%D7%9C%D7%9B%D7%99%D7%AA%D7%AA_%D7%9B%D7%95%D7%A0%D7%A0%D7%95%D7%AA_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91" title="התנדבות לכיתת כוננות ביישוב">התנדבות לכיתת כוננות ביישוב</a></li>
<li><a href="/he/%D7%A4%D7%A0%D7%99%D7%99%D7%94_%D7%9C%D7%A8%D7%A9%D7%95%D7%AA_%D7%94%D7%9E%D7%A7%D7%95%D7%9E%D7%99%D7%AA_%D7%91%D7%A0%D7%95%D7%A9%D7%90_%D7%9B%D7%99%D7%AA%D7%AA_%D7%9B%D7%95%D7%A0%D7%A0%D7%95%D7%AA_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91" title="פנייה לרשות המקומית בנושא כיתת כוננות ביישוב">פנייה לרשות המקומית בנושא כיתת כוננות ביישוב</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%A0%D7%A2%D7%93%D7%A8%D7%95_%D7%9E%D7%A2%D7%91%D7%95%D7%93%D7%AA%D7%9D_%D7%9C%D7%A6%D7%95%D7%A8%D7%9A_%D7%94%D7%A9%D7%92%D7%97%D7%94_%D7%A2%D7%9C_%D7%99%D7%9C%D7%93%D7%99%D7%94%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="איסור פיטורי עובדים שנעדרו מעבודתם לצורך השגחה על ילדיהם במהלך מלחמת חרבות ברזל">איסור פיטורי בני זוג של אנשי כוחות הביטחון וההצלה ושותפיהם להורות שנעדרו מעבודתם לצורך השגחה על ילדיהם</a></li>
<li><a href="/he/%D7%A6%D7%91%D7%99%D7%A8%D7%AA_%D7%99%D7%9E%D7%99_%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%A9%D7%9C%D7%90_%D7%99%D7%9B%D7%9C%D7%95_%D7%9C%D7%A0%D7%A6%D7%9C_%D7%90%D7%AA_%D7%99%D7%9E%D7%99_%D7%94%D7%97%D7%95%D7%A4%D7%A9%D7%94_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל">צבירת ימי חופשה לעובדים שלא יכלו לנצל את ימי החופשה עקב מלחמת חרבות ברזל</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="אנשים_עם_מוגבלות"></span><span class="mw-headline" id=".D7.90.D7.A0.D7.A9.D7.99.D7.9D_.D7.A2.D7.9D_.D7.9E.D7.95.D7.92.D7.91.D7.9C.D7.95.D7.AA">אנשים עם מוגבלות</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/about/news/Pages/HaarachatMoedTviaLekizbaVeirur.aspx">דחיית המועד האחרון להגשת תביעה לקצבה מהביטוח הלאומי או ערעור לוועדה רפואית</a></li>
<li><a href="/he/%D7%A1%D7%99%D7%95%D7%A2_%D7%91%D7%94%D7%A9%D7%90%D7%9C%D7%94_%D7%95%D7%91%D7%A8%D7%9B%D7%99%D7%A9%D7%AA_%D7%A2%D7%96%D7%A8%D7%99%D7%9D_%D7%9C%D7%93%D7%99%D7%95%D7%A8_%D7%9C%D7%90%D7%A0%D7%A9%D7%99%D7%9D_%D7%A2%D7%9D_%D7%9E%D7%95%D7%92%D7%91%D7%9C%D7%95%D7%AA_%D7%A9%D7%A4%D7%95%D7%A0%D7%95_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="סיוע בהשאלה וברכישת עזרים לדיור לאנשים עם מוגבלות שפונו מבתיהם במלחמת חרבות ברזל">סיוע בהשאלה וברכישת עזרים לדיור לאנשים עם מוגבלות שפונו מבתיהם</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/HaravotBarzel1/KizbeotHB/Pages/Nechut_HB.aspx">הארכה אוטומטית של זכאות זמנית לקצבאות נכות כללית, שירותים מיוחדים וילד נכה</a> באתר הביטוח הלאומי</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/service/emotional-therapy-or-group-intervention-due-to-the-corona-crisis">קבלת טיפול רגשי בעקבות המשבר</a> - לאנשים עם מוגבלות, בני משפחתם והצוות המטפל</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/topics/molsa-people-with-disabilities-iron-swords/govil-landing-page">מידע שימושי לאנשים עם מוגבלות ובני משפחותיהם</a> באתר משרד הרווחה</li>
<li>מידע באתר נציבות שוויון זכויות לאנשים עם מוגבלות:
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/guides/general-emergency-disabilities">הנחיות חירום כלליות לאנשים עם מוגבלות</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/guides/emergency-sign-language">הנחיות חירום בשפת הסימנים</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/guides/emergency-mobility">הנחיות חירום לאנשים עם מוגבלות בניידות</a></li></ul></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.aisrael.org/?CategoryID=2741">מידע מסייע לאנשים עם מוגבלות בשעת חירום</a> באתר נגישות ישראל</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.molsa.gov.il/Focus/Documents/%D7%97%D7%95%D7%91%D7%A8%D7%AA%20%D7%94%D7%A0%D7%97%D7%99%D7%95%D7%AA%20%D7%9C%D7%97%D7%99%D7%A8%D7%95%D7%9D%20%D7%91%D7%A4%D7%99%D7%A9%D7%95%D7%98%20%D7%9C%D7%A9%D7%95%D7%A0%D7%99.pdf">מה אני צריך לעשות בזמן אזעקה או "צבע אדום"?</a> - מידע מונגש בלשון פשוטה למתקשים בהבנת השפה</li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.oref.org.il/12410-he/Pakar.aspx?parentCategory=12838">ירי רקטות וטילים - מידע לאנשים עם מוגבלות וקשישים</a> באתר פיקוד העורף</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://ironswords.health.gov.il/rights-of-evacuated-persons/mobility-rehabilitation-aids/">מכשירי שיקום וניידות למי שפונו או התפנו מביתם ללא המכשיר</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="מבוגרים,_בני_משפחה_מטפלים,_ניצולי_שואה,_ומטופלים_סיעודיים"></span><span class="mw-headline" id=".D7.9E.D7.91.D7.95.D7.92.D7.A8.D7.99.D7.9D.2C_.D7.91.D7.A0.D7.99_.D7.9E.D7.A9.D7.A4.D7.97.D7.94_.D7.9E.D7.98.D7.A4.D7.9C.D7.99.D7.9D.2C_.D7.A0.D7.99.D7.A6.D7.95.D7.9C.D7.99_.D7.A9.D7.95.D7.90.D7.94.2C_.D7.95.D7.9E.D7.98.D7.95.D7.A4.D7.9C.D7.99.D7.9D_.D7.A1.D7.99.D7.A2.D7.95.D7.93.D7.99.D7.99.D7.9D">מבוגרים, בני משפחה מטפלים, ניצולי שואה, ומטופלים סיעודיים</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/guides/molsa-senior-citizens-iron-swords-information?chapterIndex=1">מידע לאזרחים ותיקים ובני משפחה מטפלים</a> באתר משרד הרווחה</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/news301023_4">הארכה ב-3 חודשים של תוקף הרישיונות לעובדים הזרים וההיתרים למעסיקים</a> - באתר רשות האוכלוסין וההגירה</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/BlobFolder/news/news241023_1/he/relief_nursing_industry_only_war_of_iron_swords.pdf">הקלות באישורים לעובדים זרים ומוקד למעסיקים מיישובים מפונים בצפון ובדרום</a>  - באתר רשות האוכלוסין וההגירה</li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.facebook.com/photo/?fbid=657068483232447&amp;set=a.510196087919688">העברת סיעודיים ומטפליהם מיישובים מפונים למוסדות גריאטריים, בתי דיור מוגן וכו'</a> - עמותת מגיני בתי"ה</li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%97%D7%93_%D7%A4%D7%A2%D7%9E%D7%99_%D7%9C%D7%A0%D7%99%D7%A6%D7%95%D7%9C%D7%99_%D7%A9%D7%95%D7%90%D7%94_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מענק חד פעמי לניצולי שואה עקב מלחמת חרבות ברזל">מענק חד פעמי לניצולי שואה עקב מלחמת חרבות ברזל</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="נשים"></span><span class="mw-headline" id=".D7.A0.D7.A9.D7.99.D7.9D">נשים</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%90%D7%A4%D7%9C%D7%99%D7%99%D7%AA_%D7%A2%D7%95%D7%91%D7%93_%D7%9E%D7%98%D7%A2%D7%9E%D7%99_%D7%9E%D7%99%D7%9F" title="איסור אפליית עובד מטעמי מין">איסור אפליית עובדת מטעמי מין</a></li>
<li><a href="/he/%D7%A9%D7%90%D7%9C%D7%95%D7%AA_%D7%95%D7%AA%D7%A9%D7%95%D7%91%D7%95%D7%AA_%D7%9C%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D_%D7%95%D7%9C%D7%9E%D7%A2%D7%A1%D7%99%D7%A7%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C#.D7.94.D7.90.D7.9D_.D7.9E.D7.95.D7.AA.D7.A8_.D7.9C.D7.A4.D7.98.D7.A8_.D7.A2.D7.95.D7.91.D7.93_.D7.A9.D7.A0.D7.A2.D7.93.D7.A8_.D7.9E.D7.A2.D7.91.D7.95.D7.93.D7.AA.D7.95_.D7.A2.D7.9C_.D7.9E.D7.A0.D7.AA_.D7.9C.D7.94.D7.A9.D7.92.D7.99.D7.97_.D7.A2.D7.9C_.D7.99.D7.9C.D7.93.D7.99.D7.95_.D7.A2.D7.A7.D7.91_.D7.A1.D7.92.D7.99.D7.A8.D7.AA_.D7.9E.D7.A2.D7.A8.D7.9B.D7.AA_.D7.94.D7.97.D7.99.D7.A0.D7.95.D7.9A_.D7.91.D7.94.D7.AA.D7.90.D7.9D_.D7.9C.D7.94.D7.95.D7.A8.D7.90.D7.95.D7.AA_.D7.A4.D7.99.D7.A7.D7.95.D7.93_.D7.94.D7.A2.D7.95.D7.A8.D7.A3.3F" title="שאלות ותשובות לעובדים ולמעסיקים במהלך מלחמת חרבות ברזל">אמהות שנעדרו מעבודה לצורך השגחה על ילדיהם</a></li>
<li><a href="/he/%D7%A9%D7%A2%D7%AA_%D7%94%D7%99%D7%A2%D7%93%D7%A8%D7%95%D7%AA_%D7%9C%D7%A2%D7%95%D7%91%D7%93/%D7%AA_%D7%91%D7%92%D7%99%D7%9F_%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%A9%D7%9C_%D7%91%D7%9F/%D7%91%D7%AA_%D7%94%D7%96%D7%95%D7%92" title="שעת היעדרות לעובד/ת בגין שירות מילואים של בן/בת הזוג">שעת היעדרות מהעבודה לבנות זוג של משרתי מילואים</a></li>
<li><a href="/he/%D7%9E%D7%93%D7%A8%D7%99%D7%9A_%D7%9C%D7%94%D7%95%D7%A8%D7%99%D7%9D_%D7%99%D7%97%D7%99%D7%93%D7%A0%D7%99%D7%99%D7%9D_(%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%A9%D7%91%D7%A8%D7%90%D7%A9%D7%9F_%D7%94%D7%95%D7%A8%D7%94_%D7%A2%D7%A6%D7%9E%D7%90%D7%99)_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מדריך להורים יחידניים (משפחות שבראשן הורה עצמאי) במלחמת חרבות ברזל">מדריך לאמהות יחידניות במלחמה</a></li>
<li><a href="/he/%D7%91%D7%A7%D7%A9%D7%94_%D7%9C%D7%94%D7%99%D7%AA%D7%A8_%D7%9C%D7%A4%D7%99%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%90%D7%95_%D7%A4%D7%92%D7%99%D7%A2%D7%94_%D7%91%D7%94%D7%99%D7%A7%D7%A3_%D7%9E%D7%A9%D7%A8%D7%94_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%A7_%D7%A2%D7%91%D7%95%D7%93%D7%AA_%D7%A0%D7%A9%D7%99%D7%9D" title="בקשה להיתר לפיטורים או פגיעה בהיקף משרה לפי חוק עבודת נשים">בקשה להיתר לפיטורים או הוצאה לחל"ת</a> עבור:
<ul><li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%92%D7%99%D7%A2%D7%94_%D7%91%D7%94%D7%99%D7%A7%D7%A3_%D7%9E%D7%A9%D7%A8%D7%94_%D7%90%D7%95_%D7%91%D7%94%D7%9B%D7%A0%D7%A1%D7%94_%D7%A9%D7%9C_%D7%90%D7%99%D7%A9%D7%94_%D7%91%D7%94%D7%99%D7%A8%D7%99%D7%95%D7%9F" title="איסור פגיעה בהיקף משרה או בהכנסה של אישה בהיריון">נשים בהריון</a></li>
<li><a href="/he/%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%AA%D7%A2%D7%A1%D7%95%D7%A7%D7%94_%D7%9C%D7%90%D7%97%D7%A8_%D7%94%D7%A4%D7%9C%D7%94" title="זכויות תעסוקה לאחר הפלה">נשים שעברו הפלה</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%92%D7%99%D7%A2%D7%94_%D7%91%D7%94%D7%99%D7%A7%D7%A3_%D7%9E%D7%A9%D7%A8%D7%94_%D7%90%D7%95_%D7%94%D7%9B%D7%A0%D7%A1%D7%94_%D7%A9%D7%9C_%D7%A2%D7%95%D7%91%D7%93/%D7%AA_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%98%D7%99%D7%A4%D7%95%D7%9C%D7%99_%D7%A4%D7%95%D7%A8%D7%99%D7%95%D7%AA" title="איסור פגיעה בהיקף משרה או הכנסה של עובד/ת במהלך טיפולי פוריות">נשים בטיפולי פוריות</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%92%D7%99%D7%A2%D7%94_%D7%91%D7%94%D7%99%D7%A7%D7%A3_%D7%9E%D7%A9%D7%A8%D7%94_%D7%90%D7%95_%D7%91%D7%94%D7%9B%D7%A0%D7%A1%D7%94_%D7%A9%D7%9C_%D7%A2%D7%95%D7%91%D7%93%D7%AA_%D7%90%D7%95_%D7%A2%D7%95%D7%91%D7%93_%D7%94%D7%A0%D7%9E%D7%A6%D7%90/%D7%AA_%D7%91%D7%97%D7%95%D7%A4%D7%A9%D7%AA_%D7%9C%D7%99%D7%93%D7%94_%D7%95%D7%9C%D7%90%D7%97%D7%A8%D7%99%D7%94" title="איסור פגיעה בהיקף משרה או בהכנסה של עובדת או עובד הנמצא/ת בחופשת לידה ולאחריה">נשים בחופשת לידה או אחריה</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%92%D7%99%D7%A2%D7%94_%D7%91%D7%94%D7%99%D7%A7%D7%A3_%D7%9E%D7%A9%D7%A8%D7%94_%D7%90%D7%95_%D7%91%D7%94%D7%9B%D7%A0%D7%A1%D7%94_%D7%A9%D7%9C_%D7%A2%D7%95%D7%91%D7%93/%D7%AA_%D7%91%D7%97%D7%9C%22%D7%AA_%D7%9C%D7%90%D7%97%D7%A8_%D7%97%D7%95%D7%A4%D7%A9%D7%AA_%D7%9C%D7%99%D7%93%D7%94_%D7%95%D7%9C%D7%90%D7%97%D7%A8%D7%99%D7%94" title="איסור פגיעה בהיקף משרה או בהכנסה של עובד/ת בחל&quot;ת לאחר חופשת לידה ולאחריה">נשים בחל"ת לאחר חופשת לידה ולאחריה</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%92%D7%99%D7%A2%D7%94_%D7%91%D7%94%D7%99%D7%A7%D7%A3_%D7%9E%D7%A9%D7%A8%D7%94_%D7%90%D7%95_%D7%91%D7%94%D7%9B%D7%A0%D7%A1%D7%94_%D7%A9%D7%9C_%D7%A2%D7%95%D7%91%D7%93%D7%AA_%D7%94%D7%A9%D7%95%D7%94%D7%94_%D7%91%D7%9E%D7%A7%D7%9C%D7%98_%D7%9C%D7%A0%D7%A9%D7%99%D7%9D_%D7%A0%D7%A4%D7%92%D7%A2%D7%95%D7%AA_%D7%90%D7%9C%D7%99%D7%9E%D7%95%D7%AA_(%D7%A0%D7%A9%D7%99%D7%9D_%D7%9E%D7%95%D7%9B%D7%95%D7%AA)" title="איסור פגיעה בהיקף משרה או בהכנסה של עובדת השוהה במקלט לנשים נפגעות אלימות (נשים מוכות)">נשים ששהו במקלט לנשים נפגעת אלימות</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%92%D7%99%D7%A2%D7%94_%D7%91%D7%94%D7%99%D7%A7%D7%A3_%D7%9E%D7%A9%D7%A8%D7%94_%D7%90%D7%95_%D7%91%D7%94%D7%9B%D7%A0%D7%A1%D7%94_%D7%A9%D7%9C_%D7%94%D7%95%D7%A8%D7%94_%D7%9E%D7%99%D7%95%D7%A2%D7%93_%D7%9C%D7%A4%D7%99_%D7%97%D7%95%D7%A7_%D7%94%D7%A1%D7%9B%D7%9E%D7%99%D7%9D_%D7%9C%D7%A0%D7%A9%D7%99%D7%90%D7%AA_%D7%A2%D7%95%D7%91%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%94%D7%94%D7%99%D7%A8%D7%99%D7%95%D7%9F" title="איסור פגיעה בהיקף משרה או בהכנסה של הורה מיועד לפי חוק הסכמים לנשיאת עוברים במהלך ההיריון">נשים בהליך פונדקאות</a></li>
<li><a href="/he/%D7%90%D7%99%D7%A1%D7%95%D7%A8_%D7%A4%D7%92%D7%99%D7%A2%D7%94_%D7%91%D7%94%D7%99%D7%A7%D7%A3_%D7%9E%D7%A9%D7%A8%D7%94_%D7%90%D7%95_%D7%91%D7%94%D7%9B%D7%A0%D7%A1%D7%94_%D7%A9%D7%9C_%D7%A2%D7%95%D7%91%D7%93_%D7%94%D7%A2%D7%95%D7%9E%D7%93_%D7%9C%D7%A7%D7%91%D7%9C_%D7%99%D7%9C%D7%93_%D7%9C%D7%90%D7%95%D7%9E%D7%A0%D7%94" title="איסור פגיעה בהיקף משרה או בהכנסה של עובד העומד לקבל ילד לאומנה">עובד/ת לקראת קבלת ילד לאומנה</a></li></ul></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="אוכלוסיות_שונות"></span><span class="mw-headline" id=".D7.90.D7.95.D7.9B.D7.9C.D7.95.D7.A1.D7.99.D7.95.D7.AA_.D7.A9.D7.95.D7.A0.D7.95.D7.AA">אוכלוסיות שונות</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/about/news/Pages/HaarachatMoedTviaLekizbaVeirur.aspx">דחיית המועד האחרון להגשת תביעה לקצבה מהביטוח הלאומי או ערעור לוועדה רפואית</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%AA%D7%A0%D7%90%D7%99_%D7%94%D7%94%D7%AA%D7%9E%D7%97%D7%95%D7%AA_%D7%95%D7%91%D7%9E%D7%91%D7%97%D7%A0%D7%99_%D7%94%D7%94%D7%A1%D7%9E%D7%9B%D7%94_%D7%9E%D7%98%D7%A2%D7%9D_%D7%9E%D7%A9%D7%A8%D7%93_%D7%94%D7%9E%D7%A9%D7%A4%D7%98%D7%99%D7%9D_%D7%91%D7%A2%D7%A7%D7%91%D7%95%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות בתנאי ההתמחות ובמבחני ההסמכה מטעם משרד המשפטים בעקבות מלחמת חרבות ברזל">הקלות למתמחים ולנבחנים במבחני הסמכה מטעם משרד המשפטים</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/guides/molsa-senior-citizens-iron-swords-information?chapterIndex=1">מידע לאזרחים ותיקים ובני משפחה מטפלים</a> באתר משרד הרווחה</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.health.gov.il/Subjects/mental_health/Anxiety/Pages/Recommendations_for_parents.aspx">המלצות להורים להתמודדות עם תגובות חרדה אצל ילדים במצבי לחץ</a> של משרד הבריאות</li>
<li><b><a href="/he/%E1%8B%A8%E1%8A%A0%E1%8B%B3%E1%8C%8B_%E1%8C%8A%E1%8B%9C" title="የአዳጋ ጊዜ">የአዳጋ ጊዜ</a></b></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.molsa.gov.il/Focus/Documents/%D7%97%D7%95%D7%91%D7%A8%D7%AA%20%D7%94%D7%A0%D7%97%D7%99%D7%95%D7%AA%20%D7%9C%D7%97%D7%99%D7%A8%D7%95%D7%9D%20%D7%91%D7%A4%D7%99%D7%A9%D7%95%D7%98%20%D7%9C%D7%A9%D7%95%D7%A0%D7%99.pdf">מה אני צריך לעשות בזמן אזעקה או "צבע אדום"?</a> - מידע מונגש בלשון פשוטה למתקשים בהבנת השפה</li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.oref.org.il/12410-he/Pakar.aspx?parentCategory=12838">ירי רקטות וטילים - מידע לאנשים עם מוגבלות וקשישים</a> באתר פיקוד העורף</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://kids.gov.il/oref">אתר פיקוד העורף לילדים</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="עדכונים_בנושא_בריאות"></span><span class="mw-headline" id=".D7.A2.D7.93.D7.9B.D7.95.D7.A0.D7.99.D7.9D_.D7.91.D7.A0.D7.95.D7.A9.D7.90_.D7.91.D7.A8.D7.99.D7.90.D7.95.D7.AA">עדכונים בנושא בריאות</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://ironswords.health.gov.il/">מרכז המידע של משרד הבריאות לזמן מלחמת חרבות ברזל</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://call.gov.il/general_content_page/page/%D7%97%D7%A8%D7%91%D7%95%D7%AA-%D7%91%D7%A8%D7%96%D7%9C">זכאויות ושירותים רפואיים בתקופת מלחמת חרבות ברזל</a> באתר ״כל הבריאות״</li>
<li><a href="/he/%D7%9E%D7%A8%D7%9B%D7%96%D7%99_%D7%9E%D7%99%D7%A6%D7%95%D7%99_%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%91%D7%91%D7%AA%D7%99_%D7%97%D7%95%D7%9C%D7%99%D7%9D_%D7%A2%D7%91%D7%95%D7%A8_%D7%9E%D7%98%D7%95%D7%A4%D7%9C%D7%99%D7%9D_%D7%95%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D" title="מרכזי מיצוי זכויות בבתי חולים עבור מטופלים ומשפחותיהם">מרכזי מיצוי זכויות בבתי חולים עבור מטופלים ומשפחותיהם</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://call.gov.il/page/%D7%A7%D7%A0%D7%90%D7%91%D7%99%D7%A1-%D7%A8%D7%A4%D7%95%D7%90%D7%99---%D7%97%D7%A8%D7%91%D7%95%D7%AA-%D7%91%D7%A8%D7%96%D7%9C">הארכת תוקף רישיונות לשימוש בקנאביס רפואי</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://call.gov.il/page/%D7%98%D7%99%D7%A4%D7%95%D7%9C-%D7%AA%D7%A8%D7%95%D7%A4%D7%AA%D7%99---%D7%97%D7%A8%D7%91%D7%95%D7%AA-%D7%91%D7%A8%D7%96%D7%9C">רכישת תרופות כשאין אפשרות לקבל מרשם מהרופא</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/policies/dr-511888323">הארכה אוטומטית של טופס 29</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://call.gov.il/general_content_page/page/%D7%AA%D7%A8%D7%95%D7%9E%D7%95%D7%AA-%D7%97%D7%9C%D7%91-%D7%90%D7%9D---%D7%97%D7%A8%D7%91%D7%95%D7%AA-%D7%91%D7%A8%D7%96%D7%9C-">תרומת חלב אם</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://ironswords.health.gov.il/rights-of-evacuated-persons/mobility-rehabilitation-aids/">מכשירי שיקום וניידות למי שפונו או התפנו מביתם ללא המכשיר</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://call.gov.il/page/%D7%98%D7%99%D7%A4%D7%95%D7%9C-%D7%AA%D7%A8%D7%95%D7%A4%D7%AA%D7%99---%D7%97%D7%A8%D7%91%D7%95%D7%AA-%D7%91%D7%A8%D7%96%D7%9C">רכישת תרופות ותכשירים ללא מרשם</a></li>
<li>מידע ועדכונים בתקופת המלחמה באתרי <b>קופות החולים</b>:
<ul><li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.maccabi4u.co.il/new/maccabi_news/general/%d7%a2%d7%93%d7%9b%d7%95%d7%a0%d7%99%d7%9d-%d7%97%d7%9e%d7%99%d7%9d/">מכבי</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.clalit.co.il/he/october23war/Pages/default.aspx?ref=header_allsite">כללית</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.leumit.co.il/heb/Life/Leumit%20Stands%20with%20You/">לאומית</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.meuhedet.co.il/%D7%97%D7%A8%D7%91%D7%95%D7%AA-%D7%91%D7%A8%D7%96%D7%9C-%D7%9E%D7%90%D7%95%D7%97%D7%93%D7%AA-%D7%91%D7%97%D7%99%D7%A8%D7%95%D7%9D/?ref=tyto">מאוחדת</a></li></ul></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="אנשים_עם_חובות"></span><span class="mw-headline" id=".D7.90.D7.A0.D7.A9.D7.99.D7.9D_.D7.A2.D7.9D_.D7.97.D7.95.D7.91.D7.95.D7.AA">אנשים עם חובות</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A2%D7%93%D7%9B%D7%95%D7%A0%D7%99%D7%9D_%D7%97%D7%A9%D7%95%D7%91%D7%99%D7%9D_%D7%91%D7%A0%D7%95%D7%A9%D7%90_%D7%97%D7%93%D7%9C%D7%95%D7%AA_%D7%A4%D7%99%D7%A8%D7%A2%D7%95%D7%9F_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ועדכונים חשובים בנושא חדלות פירעון בתקופת מלחמת חרבות ברזל">הקלות ועדכונים חשובים בנושא חדלות פירעון</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%94%D7%95%D7%A6%D7%90%D7%94_%D7%9C%D7%A4%D7%95%D7%A2%D7%9C_%D7%95%D7%91%D7%9E%D7%A8%D7%9B%D7%96_%D7%9C%D7%92%D7%91%D7%99%D7%99%D7%AA_%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%A2%D7%A7%D7%91_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות עקב מלחמת חרבות ברזל">הקלות לחייבים בהוצאה לפועל ובמרכז לגביית קנסות</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/eca-in-emergency-2023-12-31">הודעות ועדכונים על פעילות ההוצאה לפועל והמרכז לגביית קנסות, אגרות והוצאות</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/eca-emergency-call-center">מוקד חירום טלפוני ייעודי לפניות בנושאי הוצאה לפועל והמרכז לגביית קנסות</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%91%D7%A0%D7%A7%D7%99%D7%9D_%D7%95%D7%91%D7%97%D7%91%D7%A8%D7%95%D7%AA_%D7%94%D7%90%D7%A9%D7%A8%D7%90%D7%99_%D7%9C%D7%AA%D7%95%D7%A9%D7%91%D7%99_%D7%94%D7%93%D7%A8%D7%95%D7%9D,_%D7%9C%D7%9E%D7%A4%D7%95%D7%A0%D7%99%D7%9D_%D7%9E%D7%91%D7%AA%D7%99%D7%94%D7%9D,_%D7%9C%D7%9E%D7%A9%D7%A8%D7%AA%D7%99_%D7%94%D7%9E%D7%99%D7%9C%D7%95%D7%90%D7%99%D7%9D_%D7%95%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%94%D7%94%D7%A8%D7%95%D7%92%D7%99%D7%9D,_%D7%94%D7%97%D7%98%D7%95%D7%A4%D7%99%D7%9D_%D7%95%D7%94%D7%A0%D7%A2%D7%93%D7%A8%D7%99%D7%9D_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" class="mw-redirect" title="הקלות בבנקים ובחברות האשראי לתושבי הדרום, למפונים מבתיהם, למשרתי המילואים ולמשפחות ההרוגים, החטופים והנעדרים במלחמת חרבות ברזל">הקלות בבנקים ובחברות האשראי לתושבי הדרום, למפונים מבתיהם, למשרתי המילואים ולמשפחות ההרוגים, החטופים והנעדרים</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%91%D7%91%D7%A0%D7%A7%D7%99%D7%9D_%D7%95%D7%97%D7%91%D7%A8%D7%95%D7%AA_%D7%9B%D7%A8%D7%98%D7%99%D7%A1%D7%99_%D7%94%D7%90%D7%A9%D7%A8%D7%90%D7%99_%D7%9C%D7%9B%D7%9C%D7%9C_%D7%94%D7%90%D7%95%D7%9B%D7%9C%D7%95%D7%A1%D7%99%D7%99%D7%94_%D7%91%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות בבנקים וחברות כרטיסי האשראי לכלל האוכלוסייה במלחמת חרבות ברזל">הקלות בבנקים וחברות כרטיסי האשראי לכלל האוכלוסייה</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%95%D7%A4%D7%98%D7%95%D7%A8%D7%99%D7%9D_%D7%9C%D7%97%D7%99%D7%99%D7%91%D7%99%D7%9D_%D7%91%D7%A7%D7%A0%D7%A1%D7%95%D7%AA_%D7%91%D7%AA%D7%97%D7%95%D7%9D_%D7%94%D7%AA%D7%9B%D7%A0%D7%95%D7%9F_%D7%95%D7%94%D7%91%D7%A0%D7%99%D7%99%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל">הקלות ופטורים לחייבים בקנסות בתחום התכנון והבנייה בתקופת מלחמת חרבות ברזל</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="כיתות_כוננות_ביישוב"></span><span class="mw-headline" id=".D7.9B.D7.99.D7.AA.D7.95.D7.AA_.D7.9B.D7.95.D7.A0.D7.A0.D7.95.D7.AA_.D7.91.D7.99.D7.99.D7.A9.D7.95.D7.91">כיתות כוננות ביישוב</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a href="/he/%D7%94%D7%AA%D7%A0%D7%93%D7%91%D7%95%D7%AA_%D7%9C%D7%9B%D7%99%D7%AA%D7%AA_%D7%9B%D7%95%D7%A0%D7%A0%D7%95%D7%AA_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91" title="התנדבות לכיתת כוננות ביישוב">התנדבות לכיתת כוננות ביישוב</a></li>
<li><a href="/he/%D7%A4%D7%A0%D7%99%D7%99%D7%94_%D7%9C%D7%A8%D7%A9%D7%95%D7%AA_%D7%94%D7%9E%D7%A7%D7%95%D7%9E%D7%99%D7%AA_%D7%91%D7%A0%D7%95%D7%A9%D7%90_%D7%9B%D7%99%D7%AA%D7%AA_%D7%9B%D7%95%D7%A0%D7%A0%D7%95%D7%AA_%D7%91%D7%99%D7%99%D7%A9%D7%95%D7%91" title="פנייה לרשות המקומית בנושא כיתת כוננות ביישוב">פנייה לרשות המקומית בנושא כיתת כוננות ביישוב</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="עדכונים_על_פעילות_מערכת_המשפט"></span><span class="mw-headline" id=".D7.A2.D7.93.D7.9B.D7.95.D7.A0.D7.99.D7.9D_.D7.A2.D7.9C_.D7.A4.D7.A2.D7.99.D7.9C.D7.95.D7.AA_.D7.9E.D7.A2.D7.A8.D7.9B.D7.AA_.D7.94.D7.9E.D7.A9.D7.A4.D7.98">עדכונים על פעילות מערכת המשפט</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/Departments/General/judicial_authority_in_state_of_emergency">הודעות ועדכונים על פעילות מערכת בתי המשפט</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/eca-in-emergency-07102023">הודעות ועדכונים על פעילות ההוצאה לפועל והמרכז לגביית קנסות, אגרות והוצאות</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="מוקדי_חירום"></span><span class="mw-headline" id=".D7.9E.D7.95.D7.A7.D7.93.D7.99_.D7.97.D7.99.D7.A8.D7.95.D7.9D">מוקדי חירום</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/about/news/Pages/KavYeodi1008-1432.aspx">קו חירום לסיוע מיידי לפצועים ולמשפחות שכולות - המוסד לביטוח לאומי</a> - טלפון: <span class="phonenum phonenum-unlinked">02-6269999</span></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/general/mehalev">מרכז השליטה הלאומי לטיפול במשפחות החטופים והנעדרים</a> <span class="phonenum phonenum-unlinked">*6754</span></li>
<li><a href="/he/%D7%9E%D7%95%D7%A7%D7%93%D7%99%D7%9D_%D7%9C%D7%A1%D7%99%D7%95%D7%A2_%D7%95%D7%AA%D7%9E%D7%99%D7%9B%D7%94_%D7%A8%D7%92%D7%A9%D7%99%D7%AA_%D7%9E%D7%98%D7%A2%D7%9D_%D7%A7%D7%95%D7%A4%D7%95%D7%AA_%D7%94%D7%97%D7%95%D7%9C%D7%99%D7%9D_%D7%95%D7%9E%D7%A8%D7%9B%D7%96%D7%99_%D7%97%D7%95%D7%A1%D7%9F_%D7%91%D7%A2%D7%A7%D7%91%D7%95%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מוקדים לסיוע ותמיכה רגשית מטעם קופות החולים ומרכזי חוסן בעקבות מלחמת חרבות ברזל">מוקדים לסיוע ותמיכה רגשית מטעם קופות החולים ומרכזי חוסן בעקבות מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%95%D7%A7%D7%93%D7%99%D7%9D_%D7%9C%D7%A1%D7%99%D7%95%D7%A2_%D7%A0%D7%A4%D7%A9%D7%99_%D7%9E%D7%98%D7%A2%D7%9D_%D7%A7%D7%95%D7%A4%D7%95%D7%AA_%D7%94%D7%97%D7%95%D7%9C%D7%99%D7%9D_%D7%9C%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%98%D7%A8%D7%90%D7%95%D7%9E%D7%94_%D7%95%D7%97%D7%A8%D7%93%D7%94_%D7%91%D7%A2%D7%A7%D7%91%D7%95%D7%AA_%D7%94%D7%9E%D7%A6%D7%91_%D7%94%D7%91%D7%98%D7%97%D7%95%D7%A0%D7%99" title="מוקדים לסיוע נפשי מטעם קופות החולים לנפגעי טראומה וחרדה בעקבות המצב הבטחוני">מוקדים לסיוע נפשי מטעם קופות החולים</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/19102023-02">מרכז חוסן ארצי</a> בטלפון מספר <span class="phonenum phonenum-unlinked">*5486</span></li>
<li><a href="/he/%D7%9E%D7%A8%D7%9B%D7%96_%D7%97%D7%95%D7%A1%D7%9F" title="מרכז חוסן">מרכזי חוסן</a></li>
<li>מוקד <a href="/he/%D7%A4%D7%99%D7%A7%D7%95%D7%93_%D7%94%D7%A2%D7%95%D7%A8%D7%A3" title="פיקוד העורף">פיקוד העורף</a>: <span class="phonenum phonenum-unlinked">104</span> (שיחת חינם)</li>
<li>קו סיוע טלפוני של <a href="/he/%D7%A0%D7%98%22%D7%9C" title="נט&quot;ל">נט"ל</a> לנפגעי הלם וחרדה: <span class="phonenum phonenum-unlinked">1800-363363</span></li>
<li>קו תמיכה נפשית "נפש אחת": <span class="phonenum phonenum-unlinked">*8944</span></li>
<li>מינהלת משרד החינוך לתמיכה בתלמידים, בהורים ובצוותי ההוראה: <span class="phonenum phonenum-unlinked">*6552</span>, שלוחה 5</li>
<li>ער"ן - עזרה ראשונה נפשית: <span class="phonenum phonenum-unlinked">1201</span></li>
<li><a href="/he/%D7%9E%D7%95%D7%A7%D7%93_%D7%9E%D7%A9%D7%A8%D7%93_%D7%94%D7%A2%D7%9C%D7%99%D7%99%D7%94_%D7%95%D7%94%D7%A7%D7%9C%D7%99%D7%98%D7%94_%D7%9C%D7%A1%D7%99%D7%95%D7%A2_%D7%95%D7%AA%D7%9E%D7%99%D7%9B%D7%94_%D7%A0%D7%A4%D7%A9%D7%99%D7%AA_%D7%9C%D7%A2%D7%95%D7%9C%D7%99%D7%9D" title="מוקד משרד העלייה והקליטה לסיוע ותמיכה נפשית לעולים">מוקד סיוע ותמיכה נפשית לעולים</a></li>
<li><a href="/he/%D7%9E%D7%95%D7%A7%D7%93%D7%99%D7%9D_%D7%9C%D7%A4%D7%A0%D7%99%D7%95%D7%AA_%D7%94%D7%A6%D7%99%D7%91%D7%95%D7%A8_%D7%95%D7%9C%D7%91%D7%99%D7%A8%D7%95%D7%A8_%D7%9E%D7%99%D7%93%D7%A2_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מוקדים לפניות הציבור ולבירור מידע במהלך מלחמת חרבות ברזל">מוקדים לפניות הציבור ולבירור מידע במהלך מלחמת חרבות ברזל</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.hamalim.org/">רשימת מוקדים וחמ"לים למידע, סיוע, התנדבות, ועוד</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/Departments/General/molsa-118">מוקד מידע וסיוע - 118</a> של משרד הרווחה והביטחון החברתי</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/news121023">מוקד חירום של רשות האוכלוסין וההגירה לתושבי עוטף עזה והסביבה</a> <span class="phonenum phonenum-unlinked">074-7086100</span></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/ics-moj">מוקד חירום של רשם הירושה למשפחות החללים והנרצחים לסיוע בבקשות לצווי ירושה וצווי קיום צוואה</a> <span class="phonenum phonenum-unlinked">077-7716411</span></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/eca-emergency-call-center">מוקד חירום טלפוני ייעודי לפניות בנושאי הוצאה לפועל והמרכז לגביית קנסות</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://govforms.gov.il/mw/forms/ReportingComplaint@moital.gov.il#!details">הרשות להגנת הצרכן ולסחר הוגן</a> - טופס לדיווח תלונות להפקעת מחיר וסחר לא הוגן - בשעת חירום</li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://arab-emergencies.org/">קואליציית ארגוני חברה אזרחית לחירום בחברה הערבית</a>  בוואטסאפ <span class="phonenum phonenum-unlinked">054-289-0209</span>  - סיוע לחברה הערבית בתקופת החירום</li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.sheba.co.il/%D7%98%D7%95%D7%A4%D7%A1/%D7%A9%D7%99%D7%91%D7%90/198899">טיפול נפשי מקוון למבוגרים וילדים</a></li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="מידע_בשפות_נוספות"></span><span class="mw-headline" id=".D7.9E.D7.99.D7.93.D7.A2_.D7.91.D7.A9.D7.A4.D7.95.D7.AA_.D7.A0.D7.95.D7.A1.D7.A4.D7.95.D7.AA">מידע בשפות נוספות</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/Departments/publications/reports/molsa-iron-swords-rights">מדריך זכויות במלחמת חרבות ברזל באתר משרד הרווחה</a> - באנגלית וצרפתית</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/Departments/news/essential-information-for-an-emergency">מידע חיוני לשעת חירום באתר משרד העלייה והקליטה</a> - בעברית</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/en/departments/news/essential-information-for-an-emergency">מידע חיוני לשעת חירום באתר משרד העלייה והקליטה</a> - באנגלית</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/fr/departments/news/essential-information-for-an-emergency">מידע חיוני לשעת חירום באתר משרד העלייה והקליטה</a> - בצרפתית</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/es/departments/news/essential-information-for-an-emergency">מידע חיוני לשעת חירום באתר משרד העלייה והקליטה</a> - בספרדית</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/ru/departments/news/essential-information-for-an-emergency">מידע חיוני לשעת חירום באתר משרד העלייה והקליטה</a> - ברוסית</li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/en/Departments/news/am-essential-information-for-an-emergency#am">מידע חיוני לשעת חירום באתר משרד העלייה והקליטה</a> - באמהרית</li></ul>
</div></div>
<div class="portal-box col-sm-4">
	<div class="portal-box-header"><h2><span id="מידע_נוסף"></span><span class="mw-headline" id=".D7.9E.D7.99.D7.93.D7.A2_.D7.A0.D7.95.D7.A1.D7.A3">מידע נוסף</span></h2></div>
	<div class="portal-box-inner">
<ul><li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/about/news/Pages/HaarachatMoedTviaLekizbaVeirur.aspx">דחיית המועד האחרון להגשת תביעה לקצבה מהביטוח הלאומי או ערעור לוועדה רפואית</a></li>
<li><a href="/he/%D7%9E%D7%A1%D7%9C%D7%95%D7%9C_%D7%9E%D7%94%D7%99%D7%A8_%D7%9C%D7%91%D7%A0%D7%99%D7%99%D7%AA_%D7%9E%D7%9E%22%D7%93_%D7%91%D7%9E%D7%91%D7%A0%D7%99%D7%9D_%D7%A0%D7%9E%D7%95%D7%9B%D7%99%D7%9D_(%D7%A4%D7%98%D7%95%D7%A8_%D7%9E%D7%94%D7%99%D7%AA%D7%A8_%D7%91%D7%A0%D7%99%D7%99%D7%94)" title="מסלול מהיר לבניית ממ&quot;ד במבנים נמוכים (פטור מהיתר בנייה)">מסלול מהיר לבניית ממ"ד במבנים נמוכים (פטור מהיתר בנייה)</a></li>
<li><a href="/he/%D7%92%D7%99%D7%95%D7%A1_%D7%9B%D7%9C%D7%99_%D7%A8%D7%9B%D7%91_%D7%A4%D7%A8%D7%98%D7%99%D7%99%D7%9D,_%D7%A6%D7%99%D7%95%D7%93_%D7%94%D7%A0%D7%93%D7%A1%D7%99_%D7%95%D7%9E%D7%AA%D7%A7%D7%A0%D7%99%D7%9D_%D7%90%D7%96%D7%A8%D7%97%D7%99%D7%99%D7%9D_%D7%91%D7%A9%D7%A2%D7%AA_%D7%97%D7%99%D7%A8%D7%95%D7%9D" title="גיוס כלי רכב פרטיים, ציוד הנדסי ומתקנים אזרחיים בשעת חירום">גיוס כלי רכב פרטיים, ציוד הנדסי ומתקנים אזרחיים בשעת חירום</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/policies/adjustments_due_to_receipt01">תעריף מופחת בחשבון המים למארחים של מפונים</a> באתר רשות המים</li>
<li><a href="/he/%D7%9E%D7%AA%D7%A0%D7%93%D7%91_%D7%A9%D7%A0%D7%A4%D7%92%D7%A2_%D7%91%D7%9E%D7%94%D7%9C%D7%9A_%D7%94%D7%AA%D7%A0%D7%93%D7%91%D7%95%D7%AA%D7%95#.D7.94.D7.A4.D7.A0.D7.99.D7.94_.D7.9C.D7.94.D7.AA.D7.A0.D7.93.D7.91.D7.95.D7.AA" title="מתנדב שנפגע במהלך התנדבותו"> הקלות בהפניית אנשים להתנדבות וביטוח אוטומטי של המתנדבים</a></li>
<li><a href="/he/%D7%94%D7%95%D7%A6%D7%90%D7%AA_%D7%A8%D7%99%D7%A9%D7%99%D7%95%D7%9F_%D7%9C%D7%A0%D7%A9%D7%99%D7%90%D7%AA_%D7%A0%D7%A9%D7%A7" title="הוצאת רישיון לנשיאת נשק">הוצאת רישיון לנשיאת נשק</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%9C%D7%95%D7%AA_%D7%9C%D7%A0%D7%94%D7%92%D7%99%D7%9D_%D7%95%D7%94%D7%90%D7%A8%D7%9B%D7%AA_%D7%AA%D7%95%D7%A7%D7%A3_%D7%A8%D7%99%D7%A9%D7%99%D7%95%D7%A0%D7%95%D7%AA_%D7%A8%D7%9B%D7%91_%D7%95%D7%A0%D7%94%D7%99%D7%92%D7%94_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="הקלות לנהגים והארכת תוקף רישיונות רכב ונהיגה בתקופת מלחמת חרבות ברזל">הקלות לנהגים והארכת תוקף רישיונות רכב ונהיגה</a></li>
<li><a href="/he/%D7%A9%D7%99%D7%9E%D7%95%D7%A9_%D7%91%D7%9E%D7%A7%D7%9C%D7%98_%D7%91%D7%91%D7%99%D7%AA_%D7%9E%D7%A9%D7%95%D7%AA%D7%A3" title="שימוש במקלט בבית משותף">שימוש במקלט בבית משותף</a></li>
<li><a href="/he/%D7%9E%D7%A2%D7%A0%D7%A7_%D7%91%D7%99%D7%98%D7%97%D7%95%D7%A0%D7%99_%D7%9C%D7%91%D7%A0%D7%99%D7%99%D7%AA_%D7%9E%D7%9E%22%D7%93_%D7%9C%D7%AA%D7%95%D7%A9%D7%91%D7%99_%D7%A7%D7%95_%D7%94%D7%A2%D7%99%D7%9E%D7%95%D7%AA" title="מענק ביטחוני לבניית ממ&quot;ד לתושבי קו העימות">מענק ביטחוני לבניית ממ"ד לתושבי קו העימות</a></li>
<li><a href="/he/%D7%91%D7%99%D7%98%D7%95%D7%9C_%D7%94%D7%92%D7%91%D7%9C%D7%AA_%D7%97%D7%A9%D7%91%D7%95%D7%9F_%D7%91%D7%92%D7%99%D7%9F_%D7%A6%27%D7%A7_%D7%A9%D7%97%D7%96%D7%A8_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="ביטול הגבלת חשבון בגין צ&#39;ק שחזר בתקופת מלחמת חרבות ברזל">ביטול הגבלת חשבון בגין צ'ק שחזר בתקופת מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%94%D7%97%D7%96%D7%A8_%D7%9B%D7%A1%D7%A4%D7%99_%D7%9C%D7%91%D7%A2%D7%9C%D7%99_%D7%9B%D7%A8%D7%98%D7%99%D7%A1_%D7%A8%D7%91-%D7%A7%D7%95_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="החזר כספי לבעלי כרטיס רב-קו בתקופת מלחמת חרבות ברזל">החזר כספי לבעלי כרטיס רב-קו בתקופת מלחמת חרבות ברזל</a></li>
<li>שאלות ותשובות באתר המועצה הישראלית לצרכנות:
<ul><li><a target="_blank" rel="noreferrer noopener" class="external text" href="http://www.consumers.org.il/category/consumer-rights-during-war">זכויות הצרכן במלחמה ובשעת חירום</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="http://www.consumers.org.il/category/flights_cancel">ביטולי טיסות</a></li></ul></li>
<li><a href="/he/%D7%94%D7%95%D7%A6%D7%90%D7%AA_%D7%93%D7%A8%D7%9B%D7%95%D7%9F_%D7%96%D7%9E%D7%A0%D7%99" title="הוצאת דרכון זמני">הוצאת דרכון זמני</a></li>
<li><a href="/he/%D7%94%D7%A0%D7%97%D7%94_%D7%90%D7%95_%D7%A4%D7%98%D7%95%D7%A8_%D7%9E%D7%AA%D7%A9%D7%9C%D7%95%D7%9D_%D7%90%D7%92%D7%A8%D7%AA_%D7%A9%D7%99%D7%A8%D7%95%D7%AA%D7%99_%D7%9B%D7%91%D7%90%D7%95%D7%AA_%D7%95%D7%94%D7%A6%D7%9C%D7%94" title="הנחה או פטור מתשלום אגרת שירותי כבאות והצלה">הנחה או פטור מתשלום אגרת שירותי כבאות והצלה</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/news/mot_updates_south_23">עדכונים ושינויים בהליכי רישוי רכב ונהיגה, תחבורה ציבורית, תעופה וכו׳) באתר משרד התחבורה</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/guides/iron-swords-updates?chapterIndex=1">מידע מתעדכן בנושא סיוע בשכר דירה, דיור ציבורי, דירה בהנחה וכו׳ באתר משרד הבינוי והשיכון</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/faq/qa_id_passport_visas">שאלות ותשובות בנושא הנפקת תעודות זהות ודרכונים, מרשם ואשרות באתר רשות האוכלוסין וההגירה</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://parents.education.gov.il/prhnet/gov-education/safety/iron-swords-qa">שאלות ותשובות בנושאי חינוך ולימודים</a></li></ul>
</div></div>
</div>
<div class="article-see-also blue-box"><div class="header">ראו גם</div><ul class="list-plain list-with-arrows"><li class="see-also-item"><a href="/he/%D7%99%D7%95%D7%9E%D7%9F_%D7%A2%D7%93%D7%9B%D7%95%D7%A0%D7%99%D7%9D_%D7%91%D7%A0%D7%95%D7%A9%D7%90_%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%91%D7%AA%D7%A7%D7%95%D7%A4%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="יומן עדכונים בנושא זכויות בתקופת מלחמת חרבות ברזל">יומן עדכונים בנושא זכויות בתקופת מלחמת חרבות ברזל</a></li><li class="see-also-item"><a href="/he/%E1%8B%A8%E1%8A%A0%E1%8B%B3%E1%8C%8B_%E1%8C%8A%E1%8B%9C" title="የአዳጋ ጊዜ">የአዳጋ ጊዜ</a></li><li class="see-also-item"><a target="_blank" rel="noopener" class="external text" href="https://www.gov.il/he/departments/gov-swords-of-iron/govil-landing-page">מידע לציבור ממשרדי הממשלה</a></li><li class="see-also-item"><a href="/he/%D7%AA%D7%A2%D7%A1%D7%95%D7%A7%D7%94_%D7%95%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%A2%D7%95%D7%91%D7%93%D7%99%D7%9D" title="תעסוקה וזכויות עובדים">תעסוקה וזכויות עובדים</a></li><li class="see-also-item"><a href="/he/%D7%9E%D7%A6%D7%91_%D7%97%D7%99%D7%A8%D7%95%D7%9D_%D7%95%D7%9E%D7%A6%D7%91_%D7%9E%D7%99%D7%95%D7%97%D7%93_%D7%91%D7%A2%D7%95%D7%A8%D7%A3" title="מצב חירום ומצב מיוחד בעורף">מצב חירום ומצב מיוחד בעורף</a></li></ul></div>
<h2><span id="גורמים_מסייעים"></span><span class="mw-headline" id=".D7.92.D7.95.D7.A8.D7.9E.D7.99.D7.9D_.D7.9E.D7.A1.D7.99.D7.99.D7.A2.D7.99.D7.9D">גורמים מסייעים</span></h2>
<div id="helpme-section" class="help-wrapper-all pagequality-ignore">
<h3><span id="מוקדים_ממשלתיים"></span><span class="mw-headline" id=".D7.9E.D7.95.D7.A7.D7.93.D7.99.D7.9D_.D7.9E.D7.9E.D7.A9.D7.9C.D7.AA.D7.99.D7.99.D7.9D">מוקדים ממשלתיים</span></h3>
<div class="national-centers"><table class="kz-help-table plainlinks"><tbody><tr><th>שם</th><th>אתר</th><th>דוא&quot;ל</th><th><span class="hidden-xs">טלפון</span><span class="visible-xs">טל&#39;</span></th></tr><tr><td class="help-name"><a href="/he/%D7%9E%D7%A9%D7%A8%D7%93_%D7%94%D7%91%D7%99%D7%98%D7%97%D7%95%D7%9F_-_%D7%90%D7%92%D7%A3_%D7%A9%D7%99%D7%A7%D7%95%D7%9D_%D7%A0%D7%9B%D7%99%D7%9D">אגף שיקום נכים במשרד הביטחון</a> </td><td class="help-site"><a href="https://shikum.mod.gov.il/contactus/pages/default.aspx" target="_blank" aria-label="למעבר לאתר חיצוני" title="למעבר לאתר חיצוני"><span class="fa-stack" aria-hidden="true">                      <i class="fa fa-circle fa-stack-2x"></i>                      <i class="fa kzi kzi-link fa-inverse fa-stack-1x"></i>                    </span><span class="text visible-xs-inline">אתר</span></a></td><td class="help-mail"></td><td class="help-phone"><div class="help-phone-btn"><i class="kzi kzi-phone" aria-hidden="true"></i><a class="phonenum" href="tel:*8150">*8150</a></div></td></tr><tr><td class="help-name"><a href="/he/%D7%A8%D7%A9%D7%95%D7%AA_%D7%94%D7%9E%D7%A1%D7%99%D7%9D_-_%D7%94%D7%A7%D7%A8%D7%9F_%D7%9C%D7%A4%D7%99%D7%A6%D7%95%D7%99%D7%99%D7%9D">מס רכוש</a> </td><td class="help-site"><a href="https://www.gov.il/he/departments/general/compensation_fund_reception" target="_blank" aria-label="למעבר לאתר חיצוני" title="למעבר לאתר חיצוני"><span class="fa-stack" aria-hidden="true">                      <i class="fa fa-circle fa-stack-2x"></i>                      <i class="fa kzi kzi-link fa-inverse fa-stack-1x"></i>                    </span><span class="text visible-xs-inline">אתר</span></a></td><td class="help-mail"></td><td class="help-phone"><div class="help-phone-btn"><i class="kzi kzi-phone" aria-hidden="true"></i><a class="phonenum" href="tel:*4954">*4954</a></div></td></tr><tr><td class="help-name"><a href="/he/%D7%A4%D7%99%D7%A7%D7%95%D7%93_%D7%94%D7%A2%D7%95%D7%A8%D7%A3">פיקוד העורף</a> </td><td class="help-site"><a href="http://www.oref.org.il/" target="_blank" aria-label="למעבר לאתר חיצוני" title="למעבר לאתר חיצוני"><span class="fa-stack" aria-hidden="true">                      <i class="fa fa-circle fa-stack-2x"></i>                      <i class="fa kzi kzi-link fa-inverse fa-stack-1x"></i>                    </span><span class="text visible-xs-inline">אתר</span></a></td><td class="help-mail"></td><td class="help-phone"><div class="help-phone-btn"><i class="kzi kzi-phone" aria-hidden="true"></i><a class="phonenum" href="tel:104">104</a></div></td></tr><tr><td class="help-name"> מרכז השליטה הלאומי לטיפול במשפחות החטופים והנעדרים</td><td class="help-site"><a href="https://www.gov.il/he/departments/general/mehalev" target="_blank" aria-label="למעבר לאתר חיצוני" title="למעבר לאתר חיצוני"><span class="fa-stack" aria-hidden="true">                      <i class="fa fa-circle fa-stack-2x"></i>                      <i class="fa kzi kzi-link fa-inverse fa-stack-1x"></i>                    </span><span class="text visible-xs-inline">אתר</span></a></td><td class="help-mail"><a href="mailto:Mashlat@pmo.gov.il" target="_blank" aria-label="לשליחת דוא&quot;ל" title="לשליחת דוא&quot;ל"><i class="kzi kzi-email" aria-hidden="true"></i><span class="text visible-xs-inline">דוא&quot;ל</span></a></td><td class="help-phone"><div class="help-phone-btn"><i class="kzi kzi-phone" aria-hidden="true"></i><a class="phonenum" href="tel:*6754">*6754</a></div></td></tr><tr><td class="help-name"> משרד המשפטים- הכוונה בסוגיות משפטיות בעקבות מצב החירום</td><td class="help-site"><a href="https://www.gov.il/he/departments/general/information_center" target="_blank" aria-label="למעבר לאתר חיצוני" title="למעבר לאתר חיצוני"><span class="fa-stack" aria-hidden="true">                      <i class="fa fa-circle fa-stack-2x"></i>                      <i class="fa kzi kzi-link fa-inverse fa-stack-1x"></i>                    </span><span class="text visible-xs-inline">אתר</span></a></td><td class="help-mail"><a href="mailto:mokedmishpati@justice.gov.il" target="_blank" aria-label="לשליחת דוא&quot;ל" title="לשליחת דוא&quot;ל"><i class="kzi kzi-email" aria-hidden="true"></i><span class="text visible-xs-inline">דוא&quot;ל</span></a></td><td class="help-phone"><div class="help-phone-btn"><i class="kzi kzi-phone" aria-hidden="true"></i><a class="phonenum" href="tel:0733928666">0733928666</a></div></td></tr><tr><td class="help-name"> קו חירום לפצועים ולמשפחות שכולות (הביטוח הלאומי)</td><td class="help-site"><a href="https://www.btl.gov.il/about/news/Pages/KavYeodi1008-1432.aspx" target="_blank" aria-label="למעבר לאתר חיצוני" title="למעבר לאתר חיצוני"><span class="fa-stack" aria-hidden="true">                      <i class="fa fa-circle fa-stack-2x"></i>                      <i class="fa kzi kzi-link fa-inverse fa-stack-1x"></i>                    </span><span class="text visible-xs-inline">אתר</span></a></td><td class="help-mail"></td><td class="help-phone"><div class="help-phone-btn"><i class="kzi kzi-phone" aria-hidden="true"></i><a class="phonenum" href="tel:02-6269999">02-6269999</a></div></td></tr><tr><td class="help-name"><a href="/he/%D7%A7%D7%95%D7%9C_%D7%94%D7%91%D7%A8%D7%99%D7%90%D7%95%D7%AA_-_%D7%9E%D7%95%D7%A7%D7%93_%D7%94%D7%A9%D7%99%D7%A8%D7%95%D7%AA_%D7%94%D7%98%D7%9C%D7%A4%D7%95%D7%A0%D7%99_%D7%A9%D7%9C_%D7%9E%D7%A9%D7%A8%D7%93_%D7%94%D7%91%D7%A8%D7%99%D7%90%D7%95%D7%AA_5400*">קול הבריאות (משרד הבריאות)</a> </td><td class="help-site"><a href="https://www.gov.il/he/Departments/General/health_call_center" target="_blank" aria-label="למעבר לאתר חיצוני" title="למעבר לאתר חיצוני"><span class="fa-stack" aria-hidden="true">                      <i class="fa fa-circle fa-stack-2x"></i>                      <i class="fa kzi kzi-link fa-inverse fa-stack-1x"></i>                    </span><span class="text visible-xs-inline">אתר</span></a></td><td class="help-mail"><a href="https://govforms.gov.il/mw/forms/callhabriut@health.gov.il#!personalInformtaion" target="_blank">פניה מקוונת</a></td><td class="help-phone"><div class="help-phone-btn"><i class="kzi kzi-phone" aria-hidden="true"></i><a class="phonenum" href="tel:*5400">*5400</a></div></td></tr></tbody></table></div></div>
<h3><span id="ארגוני_סיוע"></span><span class="mw-headline" id=".D7.90.D7.A8.D7.92.D7.95.D7.A0.D7.99_.D7.A1.D7.99.D7.95.D7.A2">ארגוני סיוע</span></h3>
<ul><li class="mw-empty-elt"></li>
<li>זכות בקלות - מידע וסיוע אישי בנושא זכויות ושירותים עקב המצב הביטחוני - בטלפון <span class="phonenum phonenum-unlinked">*5922</span> או באמצעות <a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.zbekalut.org.il/opening-an-application">פנייה מקוונת</a></li>
<li><a href="/he/%D7%9E%D7%A8%D7%9B%D7%96%D7%99_%D7%9E%D7%99%D7%A6%D7%95%D7%99_%D7%96%D7%9B%D7%95%D7%99%D7%95%D7%AA_%D7%91%D7%91%D7%AA%D7%99_%D7%97%D7%95%D7%9C%D7%99%D7%9D_%D7%A2%D7%91%D7%95%D7%A8_%D7%9E%D7%98%D7%95%D7%A4%D7%9C%D7%99%D7%9D_%D7%95%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA%D7%99%D7%94%D7%9D" title="מרכזי מיצוי זכויות בבתי חולים עבור מטופלים ומשפחותיהם">מרכזי מיצוי זכויות בבתי חולים עבור מטופלים ומשפחותיהם</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.israhelp.org.il/">IsraHelp</a> - מיזם התנדבותי לפרסום יוזמות אזרחיות בתחומים שונים (איתור נעדרים, סיוע במזון ולוגיסטיקה, תמיכה נפשית, סיוע לפצועים ולמפונים ועוד)</li>
<li><a href="/he/%D7%9E%D7%95%D7%A7%D7%93%D7%99%D7%9D_%D7%9C%D7%A1%D7%99%D7%95%D7%A2_%D7%95%D7%AA%D7%9E%D7%99%D7%9B%D7%94_%D7%A8%D7%92%D7%A9%D7%99%D7%AA_%D7%9E%D7%98%D7%A2%D7%9D_%D7%A7%D7%95%D7%A4%D7%95%D7%AA_%D7%94%D7%97%D7%95%D7%9C%D7%99%D7%9D_%D7%95%D7%9E%D7%A8%D7%9B%D7%96%D7%99_%D7%97%D7%95%D7%A1%D7%9F_%D7%91%D7%A2%D7%A7%D7%91%D7%95%D7%AA_%D7%9E%D7%9C%D7%97%D7%9E%D7%AA_%D7%97%D7%A8%D7%91%D7%95%D7%AA_%D7%91%D7%A8%D7%96%D7%9C" title="מוקדים לסיוע ותמיכה רגשית מטעם קופות החולים ומרכזי חוסן בעקבות מלחמת חרבות ברזל">מוקדים לסיוע ותמיכה רגשית מטעם קופות החולים ומרכזי חוסן בעקבות מלחמת חרבות ברזל</a></li>
<li><a href="/he/%D7%9E%D7%A8%D7%9B%D7%96_%D7%97%D7%95%D7%A1%D7%9F" title="מרכז חוסן">מרכז חוסן</a></li>
<li><a target="_blank" rel="noopener" class="external text" href="https://www.btl.gov.il/benefits/Victims_of_Hostilities/Pages/MerkazeiSiua.aspx">מרכזי סיוע לנפגעי חרדה</a> באתר המוסד לביטוח לאומי</li>
<li><a href="/he/%D7%A0%D7%98%22%D7%9C" title="נט&quot;ל">נט"ל</a> - נפגעי טראומה על רקע לאומי</li>
<li><a href="/he/%D7%A2%D7%A8%22%D7%9F_-_%D7%A2%D7%96%D7%A8%D7%94_%D7%A8%D7%90%D7%A9%D7%95%D7%A0%D7%94_%D7%A0%D7%A4%D7%A9%D7%99%D7%AA_%D7%91%D7%98%D7%9C%D7%A4%D7%95%D7%9F" class="mw-redirect" title="ער&quot;ן - עזרה ראשונה נפשית בטלפון">ער"ן - עזרה ראשונה נפשית בטלפון</a> <span class="phonenum phonenum-unlinked">1201</span> וב<a target="_blank" rel="noreferrer noopener" class="external text" href="https://www.eran.org.il/">אינטרנט</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="http://www.yadbeyad.org.il/הפרויקטים-שלנו/">מוקד תמיכה נפשית של אגודת יד ביד</a> - <span class="phonenum phonenum-unlinked">03-6204999</span></li>
<li><a href="/he/%D7%99%D7%93_%D7%90%D7%9C%D7%99%D7%A2%D7%96%D7%A8" title="יד אליעזר">יד אליעזר</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://onefamilytogether.org/he/sample-page/">משפחה אחת</a> - לקרבנות הטרור ובני משפחותיהם</li>
<li><a href="/he/%D7%90%D7%A8%D7%92%D7%95%D7%9F_%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%A4%D7%A2%D7%95%D7%9C%D7%95%D7%AA_%D7%90%D7%99%D7%91%D7%94" title="ארגון נפגעי פעולות איבה">ארגון נפגעי פעולות איבה</a></li>
<li><a href="/he/%D7%A1%D7%9C%D7%A2_-_%D7%9E%D7%A8%D7%9B%D7%96_%D7%A1%D7%99%D7%95%D7%A2_%D7%9C%D7%A2%D7%95%D7%9C%D7%94_%D7%91%D7%9E%D7%A9%D7%91%D7%A8" title="סלע - מרכז סיוע לעולה במשבר">סלע - מרכז סיוע לעולה במשבר</a></li>
<li><a href="/he/%D7%A6%D7%A2%D7%99%D7%A8%D7%99_%D7%97%D7%91%22%D7%93_%D7%9C%D7%9E%D7%A2%D7%9F_%D7%A0%D7%A4%D7%92%D7%A2%D7%99_%D7%94%D7%98%D7%A8%D7%95%D7%A8" title="צעירי חב&quot;ד למען נפגעי הטרור">צעירי חב"ד למען נפגעי הטרור</a></li>
<li><a href="/he/%D7%A2%D7%9E%D7%9A" title="עמך">עמך</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%95%D7%90%D7%9C%D7%99%D7%A6%D7%99%D7%94_%D7%94%D7%99%D7%A9%D7%A8%D7%90%D7%9C%D7%99%D7%AA_%D7%9C%D7%98%D7%A8%D7%90%D7%95%D7%9E%D7%94" title="הקואליציה הישראלית לטראומה">הקואליציה הישראלית לטראומה</a></li>
<li><a href="/he/%D7%94%D7%A7%D7%A8%D7%9F_%D7%9C%D7%99%D7%93%D7%99%D7%93%D7%95%D7%AA" title="הקרן לידידות">הקרן לידידות</a></li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://arab-emergencies.org/">קואליציית ארגוני חברה אזרחית לחירום בחברה הערבית</a>  בוואטסאפ <span class="phonenum phonenum-unlinked">054-289-0209</span>  - סיוע לחברה הערבית בתקופת החירום</li>
<li><a target="_blank" rel="noreferrer noopener" class="external text" href="https://nefeshbarzel.makshivim-il.org/">ארגון "מקשיבים"</a> - סיוע נפשי בטלפון <span class="phonenum phonenum-unlinked">*3596</span> (שלוחה 1) וכן ליווי סוציאלי, תמיכה וטיפול פסיכולוגי למשפחות נפגעי העוטף ופעולות האיבה</li></ul>
"""

In [7]:
# Regular expression to find title and corresponding text
pattern = re.compile(r'title="([^"]+)"[^>]*>([^<]+)</a>')

# Find all matches
matches = pattern.findall(html_content)

# Create list of elements between title=" and </a>
elements = [match[1] for match in matches]

# Print the list of elements
print(elements[:20])

['רשימת שאלות ותשובות', 'מדריך למי שפונו או התפנו מבתיהם עקב המלחמה', 'מדריך לעצמאים ובעלי עסקים בתקופת מלחמת חרבות ברזל', 'מדריך לעובדים בתקופת המלחמה', 'מדריך למשרתי מילואים במלחמת חרבות ברזל', 'מדריך למשתחררים משירות מילואים במלחמת חרבות ברזל', 'זכותון למשפחות החטופים והנעדרים', 'מדריך לנפגעים במסיבת נובה ופסיידאק ובני משפחותיהם', 'מדריך להורים יחידניים (משפחות שבראשן הורה עצמאי) במלחמת חרבות ברזל', 'מדריך לעמותות ולחברות לתועלת הציבור', 'שאלות ותשובות למי שנפגעו פיזית או נפשית ולבני משפחותיהם', 'שאלות ותשובות למשפחת שכולות עקב המלחמה', 'שאלות ותשובות למי שרכושם נפגע במלחמה', 'שאלות ותשובות לעצמאים ובעלי עסקים', 'שאלות ותשובות לעובדים ולמעסיקים', 'שאלות ותשובות למי שמשרתים במילואים', 'שאלות ותשובות בנושאי צרכנות', 'מדריך למי שפונו או התפנו מבתיהם עקב מלחמת חרבות ברזל', 'מענק אבטלה למפונים שאינם עובדים ושמיצו את זכאותם לדמי אבטלה', 'מענק מיוחד למפונים בגיל 67 ומעלה שאינם עובדים']


In [8]:
relevant_pages_titles = elements[1:200]

In [11]:
iterator = create_data_subset.parse_xml_file(xml_file_path,
                            page_contains_one_of = relevant_pages_titles,
                            page_must_contain_all = ["מי זכאי"],
                            section_split = ["== מי זכאי? ==", "\n== "] )

def clean_eligibility_criteria(text):
    text = text.split("== מי זכאי? ==  \n", 1)[-1]
    # Remove text enclosed in [[ ]]
    text = re.sub(r'\[\[.*?\]\]', '', text)
    # Remove URLs enclosed in [https:// ]
    text = re.sub(r'\[https://.*?\]', '', text)
    return text

i=0
data = []
for j, (title, text, term) in enumerate(iterator):
    text = clean_eligibility_criteria(text)
    data.append({"title": title, "term": term, "text": text})
    
    i+=1
    if j < 3:
      print(f"{title=}")
      print(f"related term: {term}")
      print(text)
      # print(f"{text=}\n")

print("\n\n")
print(f"Number of related pages: {i}")

title='תג חניה לנכה ולקרוב המסיע בוגר או ילד נכה'
related term: נכי צה"ל וכוחות הביטחון
== מי זכאי? ==
* מי שבגלל מחלה (או כמה מחלות) מתקשים בהליכה ותנועותיהם מסכנות את בריאותם.
* עיוורים בעלי .
*  מי שרופא שהוסמך לכך אישר שהם עונים על '''אחד''' מאלה:
** נכותם הרפואית היא בשיעור 60% לפחות ותנועתם ללא רכב עלולה לערער את מצב בריאותם.
** הם מוגבלים בהליכה וזקוקים לכיסא גלגלים לצורך ניידותם.
** יש להם נכות ברגליים והם זקוקים בגללה לרכב כאמצעי תנועה.
{{תבנית:טיפ/קביעת נכות}}

title='חובת ביטוח פנסיוני לעובדים'
related term: הפרשות לביטוח פנסיוני ולקרן השתלמות לעובדים במילואים
== מי זכאי? == <!--יש הפניה מהפורטל "עבודה בגיל השלישי"-->
* עובדים שכירים (גברים) החל מגיל 21 ועובדות שכירות החל מגיל 20.
{{טיפ|עובדים/ות שמועסקים/ות ביותר ממקום עבודה אחד, זכאים/ות להפרשות לביטוח פנסיוני בכל אחד ממקומות העבודה, שבהם הם מועסקים/ות.}}

* עובדים ועובדות שהגיעו ל (גיל 67 לנשים ולגברים) וממשיכים/ות לעבוד, יהיו זכאים להפרשת תשלומים לביטוח פנסיוני בכל מקרה.
* עובדים ועובדות חדשים/ות שנקלטו לעבודה לאחר גיל פ

In [12]:
with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

We can see two main problems:

**1. Unexpected categories under "מי זכאי"**

For example:
1. כלל ציבור הצרכנים.
2. כל תושב ישראל החבר באחת מקופות החולים
3. כלל ציבור העובדים.

**2. The text is not human readable**

For example under [תגמול למשפחות הנספים בפעולות איבה](https://www.kolzchut.org.il/he/%D7%AA%D7%92%D7%9E%D7%95%D7%9C_%D7%9C%D7%9E%D7%A9%D7%A4%D7%97%D7%95%D7%AA_%D7%94%D7%A0%D7%A1%D7%A4%D7%99%D7%9D_%D7%91%D7%A4%D7%A2%D7%95%D7%9C%D7%95%D7%AA_%D7%90%D7%99%D7%91%D7%94)